<a href="https://colab.research.google.com/github/Ashish-Abraham/PHQ9-Malayalam/blob/main/PHQ_9Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Ashish-Abraham/PHQ9-Malayalam.git

Cloning into 'PHQ9-Malayalam'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 1.27 MiB | 13.42 MiB/s, done.
Resolving deltas: 100% (3/3), done.


# Emotion Tracker

In [ ]:
!pip install tweet-preprocessor

## Dataset Preparation

In [ ]:
emotions = [
    "admiration",
    "amusement",
    "anger",
    "annoyance",
    "approval",
    "caring",
    "confusion",
    "curiosity",
    "desire",
    "disappointment",
    "disapproval",
    "disgust",
    "embarrassment",
    "excitement",
    "fear",
    "gratitude",
    "grief",
    "joy",
    "love",
    "nervousness",
    "optimism",
    "pride",
    "realization",
    "relief",
    "remorse",
    "sadness",
    "surprise",
    "neutral"
]

import os
import time
import json
import re
from tqdm import tqdm
import pandas as pd
import numpy as np

INPUT_COLUMN = "Do you feel tired or have little energy?"
INPUT_CSV = "/content/PHQ9-Malayalam/psych_patient_emotion_data (1).csv"
OUTPUT_CSV = "/content/Custom.xlsx"



In [ ]:
# # Read CSV
import pandas as pd
df_custom = pd.read_csv(INPUT_CSV, dtype=str)



In [ ]:
import pandas as pd
import json

# 1. PASTE THE JSON OUTPUT FROM THE PREVIOUS STEP HERE
# (I have included a small sample based on our previous turn for demonstration)
json_data = [
  {
    "original": "My appetite is all over the place; I either overeat or don’t eat at all.",
    "emotion_labels": [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    "rephrasings": [
      {
        "text": "It's chaotic—some days I stuff myself, and other days I can't look at food.",
        "emotion_labels": [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
      },
      {
        "text": "I can't seem to find a middle ground with eating; it's always one extreme or the other.",
        "emotion_labels": [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
      },
      {
        "text": "My eating habits are totally erratic, swinging between binging and starving.",
        "emotion_labels": [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
      },
      {
        "text": "It's frustrating because I have no control; I'm either empty or too full.",
        "emotion_labels": [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
      },
      {
        "text": "I don't know what normal is anymore; I skip meals or eat everything in sight.",
        "emotion_labels": [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
      }
    ]
  },
]

# 3. Process the data to flatten the hierarchy
processed_rows = []

for entry in json_data:
    # Add the Original text
    row = {
        "text": entry["original"],
        "type": "original",
        "label_vector": entry["emotion_labels"]
    }
    processed_rows.append(row)

    # Add the Rephrasings (Augmented data)
    for rephrasing in entry["rephrasings"]:
        row = {
            "text": rephrasing["text"],
            "type": "augmented",
            "label_vector": rephrasing["emotion_labels"]
        }
        processed_rows.append(row)

# 4. Create the DataFrame
df = pd.DataFrame(processed_rows)

# 5. Expand the 'label_vector' list into separate columns
# This takes the list of 0s and 1s and maps them to the emotion names
labels_df = pd.DataFrame(df['label_vector'].to_list(), columns=emotions)

# Concatenate the text info with the emotion columns
final_df = pd.concat([df[['text', 'type']], labels_df], axis=1)

# 6. Display and Save
print(f"Total rows created: {len(final_df)}")
print(final_df)

# Save to CSV
final_df.to_csv("psych_patient_emotion_data.csv", index=False)
print("File saved as 'psych_patient_emotion_data.csv'")

Total rows created: 6
                                                text       type  admiration  \
0  My appetite is all over the place; I either ov...   original           0   
1  It's chaotic—some days I stuff myself, and oth...  augmented           0   
2  I can't seem to find a middle ground with eati...  augmented           0   
3  My eating habits are totally erratic, swinging...  augmented           0   
4  It's frustrating because I have no control; I'...  augmented           0   
5  I don't know what normal is anymore; I skip me...  augmented           0   

   amusement  anger  annoyance  approval  caring  confusion  curiosity  ...  \
0          0      0          1         0       0          1          0  ...   
1          0      0          1         0       0          1          0  ...   
2          0      0          1         0       0          1          0  ...   
3          0      0          1         0       0          1          0  ...   
4          0      0          

In [ ]:
from datasets import load_dataset

ds = load_dataset("mrm8488/goemotions")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


goemotions.csv:   0%|          | 0.00/42.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/211225 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'],
        num_rows: 211225
    })
})

In [ ]:
train_ds = ds['train']
df_filtered = train_ds.filter(lambda example: example['example_very_unclear'] == False)

# Alternative: you can also use the negation operator
# df_filtered = train_ds.filter(lambda example: not example['example_very_unclear'])

print(f"Original dataset size: {len(train_ds)}")
print(f"Filtered dataset size: {len(df_filtered)}")
print(f"Removed {len(train_ds) - len(df_filtered)} unclear examples")

Filter:   0%|          | 0/211225 [00:00<?, ? examples/s]

Original dataset size: 211225
Filtered dataset size: 207814
Removed 3411 unclear examples


In [ ]:
emotion_columns = ['admiration', 'amusement', 'anger', 'annoyance', 'approval',
                   'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
                   'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
                   'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
                   'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise',
                   'neutral']

In [ ]:
df_filtered_pd = df_filtered.to_pandas()

# Count positive labels for each emotion
label_counts = df_filtered_pd[emotion_columns].sum()
print("Label distribution BEFORE balancing:")
print(label_counts.sort_values(ascending=False))
print(f"\nTotal rows: {len(df_filtered_pd)}")

# Step 2: Find minimum count to balance to
min_count = label_counts.min()
print(f"\nBalancing to minimum count: {min_count}")

# Step 3: Balance the dataset
balanced_indices = []

for emotion in emotion_columns:
    # Get indices where this emotion is labeled as 1
    emotion_indices = df_filtered_pd[df_filtered_pd[emotion] == 1].index.tolist()

    # Randomly sample min_count indices
    if len(emotion_indices) > min_count:
        sampled_indices = np.random.choice(emotion_indices, size=min_count, replace=False)
    else:
        sampled_indices = emotion_indices

    balanced_indices.extend(sampled_indices)

# Remove duplicates and create balanced dataset
balanced_indices = list(set(balanced_indices))
df_filtered_balanced = df_filtered_pd.loc[balanced_indices].reset_index(drop=True)

# Step 4: Verify balance
label_counts_after = df_filtered_balanced[emotion_columns].sum()
print("\nLabel distribution AFTER balancing:")
print(label_counts_after.sort_values(ascending=False))
print(f"\nTotal rows after balancing: {len(df_filtered_balanced)}")
print(f"Removed {len(df_filtered_pd) - len(df_filtered_balanced)} rows")



Label distribution BEFORE balancing:
neutral           55298
approval          17620
admiration        17131
annoyance         13618
gratitude         11625
disapproval       11424
curiosity          9692
amusement          9245
realization        8785
optimism           8715
disappointment     8469
love               8191
anger              8084
joy                7983
confusion          7359
sadness            6758
caring             5999
excitement         5629
surprise           5514
disgust            5301
desire             3817
fear               3197
remorse            2525
embarrassment      2476
nervousness        1810
pride              1302
relief             1289
grief               673
dtype: int64

Total rows: 207814

Balancing to minimum count: 673

Label distribution AFTER balancing:
admiration        1415
approval          1284
annoyance         1238
gratitude         1174
sadness           1162
disappointment    1151
optimism          1146
joy               1078
curi

In [ ]:
import re
import numpy as np
from preprocessor import clean

# Create spelling correction dictionary from common mistakes
spelling_corrections = {
    'dont': "don't", 'didnt': "didn't", 'doesnt': "doesn't", 'wont': "won't",
    'cant': "can't", 'shouldnt': "shouldn't", 'wouldnt': "wouldn't",
    'couldnt': "couldn't", 'isnt': "isn't", 'wasnt': "wasn't", 'werent': "weren't",
    'havent': "haven't", 'hasnt': "hasn't", 'hadnt': "hadn't", 'youre': "you're",
    'theyre': "they're", 'were': "we're", 'ive': "I've", 'youve': "you've",
    'theyve': "they've", 'weve': "we've", 'im': "I'm", 'hes': "he's", 'shes': "she's",
    'its': "it's", 'thats': "that's", 'whats': "what's", 'heres': "here's",
    'theres': "there's", 'wheres': "where's", 'yall': "y'all", 'gonna': "going to",
    'wanna': "want to", 'gotta': "got to", 'lemme': "let me", 'gimme': "give me",
    'dunno': "don't know", 'kinda': "kind of", 'sorta': "sort of", 'lol': '',
    'omg': '', 'wtf': '', 'tbh': '', 'imo': '', 'imho': ''
}

def clean_text(text):
    # Remove URLs, hashtags, emojis
    text = clean(text)

    # Fix UTF-8 encoding issues
    text = text.replace('\xe2\x80\x99', "'").replace('\x27', "'")
    text = text.replace('\xe2\x80\x93', "-").replace('\xe2\x80\x94', "-")

    # Remove non-alphanumeric except punctuation
    text = re.sub(r'[^a-zA-Z0-9\s\.,!?\'\"-]', '', text)

    # Fix spelling mistakes
    words = text.split()
    words = [spelling_corrections.get(word.lower(), word) for word in words]
    text = ' '.join(words)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply text cleaning
df_filtered_balanced['text'] = df_filtered_balanced['text'].apply(clean_text)

In [ ]:
# Update df_filtered to use the balanced version
df_filtered = df_filtered_balanced

In [ ]:
# Count how many rows have more than one emotion = 1
multi_label_rows = (df_filtered_balanced[emotion_columns].sum(axis=1) > 1).sum()

print(f"Rows with more than one emotion labeled as 1: {multi_label_rows}")
percentage = multi_label_rows / len(df_filtered_balanced) * 100
print(f"Percentage: {percentage:.2f}%")
rows_with_multiple = df_filtered_balanced[df_filtered_balanced[emotion_columns].sum(axis=1) > 1]
rows_with_multiple.head()


Rows with more than one emotion labeled as 1: 7027
Percentage: 38.32%


,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
21,Spice Burger. Delicious!,edgstis,oriain,ireland,t3_add4g1,t3_add4g1,1.546864e+09,61,False,1,...,0,0,0,0,0,0,0,0,0,0
22,I needed this laugh like... damn. This comment...,eeqd1yn,mutualexasperation,breakingmom,t3_aiq20i,t1_eepovxn,1.548203e+09,72,False,0,...,1,0,0,0,0,0,0,0,1,0
23,My year old sister is still creeped out by the...,eeeotpv,MrFishpaw,netflix,t3_ahe81c,t3_ahe81c,1.547866e+09,57,False,0,...,0,0,0,0,0,0,0,0,0,0
24,Honestly at this point I don't think there's a...,ee1pf1o,GwenHarris,SuicideWatch,t3_aezjur,t1_edufjmd,1.547475e+09,42,False,0,...,0,1,0,0,1,0,0,0,0,0
25,"Iunno, looks pretty Western to me.",eec67qc,ting_bu_dong,China,t3_ah6tpn,t1_eec65fa,1.547796e+09,37,False,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split


columns_to_keep = ['text'] + emotion_columns

# Keep only the relevant columns from GoEmotions
df_goemotions_clean = df_filtered[columns_to_keep].copy()

# Step 3: Ensure your custom dataset has the same column order
df_custom_clean = df_custom[columns_to_keep].copy()

print(f"\nCustom dataset size: {len(df_custom_clean)}")
print(f"GoEmotions clean size: {len(df_goemotions_clean)}")

# Step 4: Verify column alignment
print("\nColumn alignment check:")
print(f"Custom columns: {list(df_custom_clean.columns)}")
print(f"GoEmotions columns: {list(df_goemotions_clean.columns)}")
print(f"Columns match: {list(df_custom_clean.columns) == list(df_goemotions_clean.columns)}")

# Step 5: Train-test split for custom dataset (80-20 split)
custom_train, custom_test = train_test_split(
    df_custom_clean,
    test_size=0.2,
    random_state=42
)

# Step 6: Train-test split for GoEmotions dataset (80-20 split)
goemotions_train, goemotions_test = train_test_split(
    df_goemotions_clean,
    test_size=0.2,
    random_state=42
)

# Step 7: Combine the training sets
train_combined = pd.concat([custom_train, goemotions_train], ignore_index=True)

# Step 8: Combine the test sets
test_combined = pd.concat([custom_test, goemotions_test], ignore_index=True)

# Step 9: Shuffle the combined datasets
train_combined = train_combined.sample(frac=1, random_state=42).reset_index(drop=True)
test_combined = test_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# Display summary
print("\n" + "="*60)
print("FINAL DATASET SIZES:")
print("="*60)
print(f"Training set: {len(train_combined)} samples")
print(f"  - From custom dataset: {len(custom_train)}")
print(f"  - From GoEmotions: {len(goemotions_train)}")
print(f"\nTest set: {len(test_combined)} samples")
print(f"  - From custom dataset: {len(custom_test)}")
print(f"  - From GoEmotions: {len(goemotions_test)}")
print("="*60)

# Display first few rows of combined datasets
print("\nTraining set preview:")
print(train_combined.head())
print("\nTest set preview:")
print(test_combined.head())


Custom dataset size: 270
GoEmotions clean size: 18340

Column alignment check:
Custom columns: ['text', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
GoEmotions columns: ['text', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
Columns match: True

FINAL DATASET SIZES:
Training set: 14888 samples
  - From custom dataset: 216
  - From GoEmotions: 14672

Test set: 3722 samples
  - From custom dataset: 54
  - From GoEmotions: 3

## Model Training

### MURIL

In [ ]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModel,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import f1_score, accuracy_score, hamming_loss
import torch.nn as nn
import matplotlib.pyplot as plt

# ============================================================
# HYPERPARAMETER CHOICES - DETAILED EXPLANATION
# ============================================================
"""
1. MODEL CHOICE: google/muril-base-cased
   - Pre-trained on 17 Indian languages including Malayalam
   - Better cross-lingual transfer than mBERT for Indian languages

2. LEARNING RATE: 2e-5 (classification head), 1e-5 (encoder layers)
   - Lower LR for encoder to preserve cross-lingual representations
   - Higher LR for head since it's trained from scratch

3. BATCH SIZE: 16 (with gradient accumulation steps = 2, effective = 32)
   - Balanced for GPU memory and stable gradients
   - Effective batch size 32 is optimal for transformer fine-tuning

4. EPOCHS: 5-8
   - Start with 5, use early stopping to prevent overfitting
   - Cross-lingual transfer needs careful regularization

5. WARMUP: 10% of training steps
   - Gradual learning rate increase prevents destabilizing pre-trained weights

6. WEIGHT DECAY: 0.01
   - Regularization to maintain generalization to Malayalam

7. DROPOUT: 0.3 (classification head)
   - Higher dropout for regularization during cross-lingual transfer

8. GRADIENT CLIPPING: 1.0
   - Prevents exploding gradients with multi-label BCE loss

9. FREEZING STRATEGY: Gradual unfreezing
   - Phase 1: Freeze encoder, train classification head (2 epochs)
   - Phase 2: Unfreeze top 4 encoder layers (3 epochs)
   - Preserves cross-lingual knowledge while adapting to emotions
"""

"""
MURIL Fine-tuning for Multi-Label Emotion Classification - FIXED VERSION
Addresses zero prediction issue with proper BCE loss and model architecture
"""

"""
MURIL Fine-tuning for Multi-Label Emotion Classification - FIXED VERSION
Addresses zero prediction issue with proper BCE loss and model architecture
"""

# ============================================================
# 1. SETUP AND CONFIGURATION
# ============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

MODEL_NAME = "google/muril-base-cased"
NUM_LABELS = 28
MAX_LENGTH = 128

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ============================================================
# 2. PREPARE DATASETS WITH PROPER FORMATTING
# ============================================================

class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(label)  # Float for BCE loss
        }

# Prepare data
train_texts = train_combined['text'].tolist()
train_labels = train_combined[emotion_columns].values.astype(np.float32)

test_texts = test_combined['text'].tolist()
test_labels = test_combined[emotion_columns].values.astype(np.float32)

train_dataset = EmotionDataset(train_texts, train_labels, tokenizer, MAX_LENGTH)
test_dataset = EmotionDataset(test_texts, test_labels, tokenizer, MAX_LENGTH)

# Count positives per label
num_positives = train_dataset.labels.sum(axis=0)
num_samples = len(train_dataset)
num_negatives = num_samples - num_positives
# Calculate ratio: negatives / positives
# If you have 10 positives and 90 negatives, weight should be 9.
pos_weights = torch.tensor(num_negatives / num_positives)
pos_weights = pos_weights.to(device) # Move to GPU
print(f"Calculated pos_weights: {pos_weights[:5]}...") # Check values (should be > 1)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Check label distribution
print("\nLabel distribution in training set:")
for idx, emotion in enumerate(emotion_columns[:5]):  # Show first 5
    pos_count = (train_labels[:, idx] == 1).sum()
    print(f"  {emotion}: {pos_count}/{len(train_labels)} ({pos_count/len(train_labels)*100:.1f}%)")

# ============================================================
# 3. FIXED MODEL ARCHITECTURE
# ============================================================

class MURILForMultiLabelClassification(nn.Module):
    def __init__(self, model_name, num_labels, pos_weights=None, dropout=0.1):
        super().__init__()
        self.num_labels = num_labels
        self.pos_weights = pos_weights # Store weights
        # Load MURIL base model
        self.muril = AutoModel.from_pretrained(model_name)

        # Classification head
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.muril.config.hidden_size, num_labels)

        # Initialize classifier weights properly
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(self, input_ids, attention_mask, labels=None):
        # Get MURIL outputs
        outputs = self.muril(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Use [CLS] token representation
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            # Pass pos_weight to the loss function
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.pos_weights)
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))

        return {'loss': loss, 'logits': logits}


# Initialize model
model = MURILForMultiLabelClassification(MODEL_NAME, NUM_LABELS, pos_weights=pos_weights, dropout=0.3)
model.to(device)

print(f"\nModel initialized with {sum(p.numel() for p in model.parameters())} parameters")

# ============================================================
# 4. DIAGNOSTIC: Check Initial Predictions
# ============================================================

print("\n" + "="*60)
print("DIAGNOSTIC: Initial Model Predictions (Before Training)")
print("="*60 + "\n")

model.eval()
with torch.no_grad():
    sample_batch = next(iter(torch.utils.data.DataLoader(train_dataset, batch_size=8)))
    sample_input_ids = sample_batch['input_ids'].to(device)
    sample_attention_mask = sample_batch['attention_mask'].to(device)

    outputs = model(sample_input_ids, sample_attention_mask)
    sample_logits = outputs['logits']
    sample_probs = torch.sigmoid(sample_logits)

    print(f"Logits range: [{sample_logits.min():.3f}, {sample_logits.max():.3f}]")
    print(f"Probabilities range: [{sample_probs.min():.3f}, {sample_probs.max():.3f}]")
    print(f"Mean probability: {sample_probs.mean():.3f}")
    print(f"Predictions > 0.5: {(sample_probs > 0.5).sum().item()} out of {sample_probs.numel()}")

# ============================================================
# 5. FREEZING STRATEGY - PHASE 1
# ============================================================

def freeze_encoder(model):
    """Freeze MURIL encoder, keep classification head trainable"""
    for param in model.muril.parameters():
        param.requires_grad = False
    # Ensure classifier is trainable
    for param in model.classifier.parameters():
        param.requires_grad = True
    print("✓ Encoder frozen - only classification head will be trained")

def unfreeze_top_layers(model, num_layers=4):
    """Unfreeze top N encoder layers"""
    total_layers = len(model.muril.encoder.layer)
    for i in range(total_layers - num_layers, total_layers):
        for param in model.muril.encoder.layer[i].parameters():
            param.requires_grad = True
    print(f"✓ Top {num_layers} encoder layers unfrozen")

freeze_encoder(model)

# ============================================================
# 6. IMPROVED METRICS WITH DEBUGGING
# ============================================================

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert logits to probabilities
    probs = 1 / (1 + np.exp(-logits))  # Sigmoid
    predictions = (probs > 0.5).astype(int)

    # Debug: Check prediction distribution
    num_positives = predictions.sum()
    total_predictions = predictions.size

    # Multi-label metrics
    f1_micro = f1_score(labels, predictions, average='micro', zero_division=0)
    f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)
    f1_weighted = f1_score(labels, predictions, average='weighted', zero_division=0)
    hamming = hamming_loss(labels, predictions)
    exact_match = accuracy_score(labels, predictions)

    # Additional diagnostic metrics
    avg_labels_per_sample = labels.sum(axis=1).mean()
    avg_predictions_per_sample = predictions.sum(axis=1).mean()

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'hamming_loss': hamming,
        'exact_match': exact_match,
        'num_positive_preds': num_positives,
        'avg_labels_per_sample': avg_labels_per_sample,
        'avg_preds_per_sample': avg_predictions_per_sample
    }

# ============================================================
# 7. PHASE 1 TRAINING - WITH BETTER HYPERPARAMETERS
# ============================================================

training_args_phase1 = TrainingArguments(
    output_dir='./muril_emotion_phase1',
    num_train_epochs=3,  # Increased from 2
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=1e-3,  # Changed from 1e-3 to 5e-5
    weight_decay=0.05,
    warmup_ratio=0.15,
    logging_dir='./logs_phase1',
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='f1_macro',
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    report_to="none",
    seed=42,
    label_smoothing_factor=0.0  # No label smoothing for multi-label
)

print("\n" + "="*60)
print("PHASE 1: Training Classification Head (Encoder Frozen)")
print("="*60 + "\n")

trainer_phase1 = Trainer(
    model=model,
    args=training_args_phase1,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

trainer_phase1.train()

# Evaluate Phase 1
phase1_results = trainer_phase1.evaluate()
print("\nPhase 1 Results:")
for key, value in phase1_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# ============================================================
# 8. CHECK IF MODEL IS LEARNING
# ============================================================

print("\n" + "="*60)
print("POST-PHASE 1 DIAGNOSTIC")
print("="*60 + "\n")

model.eval()
with torch.no_grad():
    sample_batch = next(iter(torch.utils.data.DataLoader(test_dataset, batch_size=8)))
    sample_input_ids = sample_batch['input_ids'].to(device)
    sample_attention_mask = sample_batch['attention_mask'].to(device)
    sample_labels = sample_batch['labels']

    outputs = model(sample_input_ids, sample_attention_mask)
    sample_logits = outputs['logits']
    sample_probs = torch.sigmoid(sample_logits)

    print(f"Logits range: [{sample_logits.min():.3f}, {sample_logits.max():.3f}]")
    print(f"Probabilities range: [{sample_probs.min():.3f}, {sample_probs.max():.3f}]")
    print(f"Mean probability: {sample_probs.mean():.3f}")
    print(f"Predictions > 0.5: {(sample_probs > 0.5).sum().item()} out of {sample_probs.numel()}")
    print(f"Actual positive labels: {sample_labels.sum().item()}")

# Only continue to Phase 2 if model is learning
if phase1_results['eval_f1_macro'] > 0.01:
    print("\n✓ Model is learning! Proceeding to Phase 2...")

    # ============================================================
    # 9. PHASE 2 - UNFREEZE TOP LAYERS
    # ============================================================

    print("\n" + "="*60)
    print("PHASE 2: Fine-tuning Top Encoder Layers")
    print("="*60 + "\n")

    unfreeze_top_layers(model, num_layers=4)

    training_args_phase2 = TrainingArguments(
        output_dir='./muril_emotion_phase2',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=2,
        learning_rate=2e-5,
        weight_decay=0.05,
        warmup_ratio=0.15,
        max_grad_norm=1.0,
        logging_dir='./logs_phase2',
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=200,
        save_strategy="steps",
        save_steps=200,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        fp16=torch.cuda.is_available(),
        dataloader_num_workers=2,
        report_to="none",
        seed=42
    )

    trainer_phase2 = Trainer(
        model=model,
        args=training_args_phase2,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

    trainer_phase2.train()
    final_results = trainer_phase2.evaluate()

else:
    print("\n⚠ WARNING: Model not learning in Phase 1!")
    print("Skipping Phase 2. Need to debug further.")
    final_results = phase1_results

# ============================================================
# 10. FINAL EVALUATION WITH ADAPTIVE THRESHOLD
# ============================================================

print("\n" + "="*60)
print("FINAL EVALUATION WITH ADAPTIVE THRESHOLD")
print("="*60 + "\n")

# Try different thresholds to find optimal
model.eval()
all_logits = []
all_labels = []

with torch.no_grad():
    for batch in torch.utils.data.DataLoader(test_dataset, batch_size=32):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask)
        all_logits.append(outputs['logits'].cpu())
        all_labels.append(batch['labels'])

all_logits = torch.cat(all_logits, dim=0)
all_labels = torch.cat(all_labels, dim=0)
all_probs = torch.sigmoid(all_logits).numpy()
all_labels = all_labels.numpy()

# Test different thresholds
print("Testing different prediction thresholds:")
for threshold in [0.3, 0.4, 0.5, 0.6, 0.7]:
    preds = (all_probs > threshold).astype(int)
    f1_macro = f1_score(all_labels, preds, average='macro', zero_division=0)
    f1_micro = f1_score(all_labels, preds, average='micro', zero_division=0)
    avg_preds = preds.sum() / len(preds)
    print(f"  Threshold {threshold:.1f} - F1 Macro: {f1_macro:.4f}, F1 Micro: {f1_micro:.4f}, Avg preds/sample: {avg_preds:.2f}")

# Use best threshold
best_threshold = 0.5
pred_labels = (all_probs > best_threshold).astype(int)

print(f"\nUsing threshold: {best_threshold}")
print("\nFinal Results:")
print(f"  F1 Macro:     {f1_score(all_labels, pred_labels, average='macro', zero_division=0):.4f}")
print(f"  F1 Micro:     {f1_score(all_labels, pred_labels, average='micro', zero_division=0):.4f}")
print(f"  F1 Weighted:  {f1_score(all_labels, pred_labels, average='weighted', zero_division=0):.4f}")
print(f"  Hamming Loss: {hamming_loss(all_labels, pred_labels):.4f}")
print(f"  Exact Match:  {accuracy_score(all_labels, pred_labels):.4f}")

# Per-emotion performance
print("\n" + "="*60)
print("PER-EMOTION PERFORMANCE")
print("="*60 + "\n")

for idx, emotion in enumerate(emotion_columns):
    f1 = f1_score(all_labels[:, idx], pred_labels[:, idx], zero_division=0)
    support = all_labels[:, idx].sum()
    avg_prob = all_probs[:, idx].mean()
    print(f"{emotion:15s} - F1: {f1:.4f}, Support: {int(support):4d}, Avg Prob: {avg_prob:.3f}")

# ============================================================
# 11. SAVE MODEL
# ============================================================
# Save the underlying MURIL model (feature extractor) separately
# so it can be loaded by AutoModel.from_pretrained in the next step.
model.muril.save_pretrained('./muril_emotion_final')

# Save the tokenizer, which is associated with the MURIL model
tokenizer.save_pretrained('./muril_emotion_final')

# If the classification head's weights are also needed later, save its state_dict
# The current plan is to use XGBoost, so this might be optional for the current flow,
# but good practice for full model recovery.
torch.save(model.classifier.state_dict(), './muril_emotion_final/classifier_head.pt')

print("\n✓ MURIL feature extractor and tokenizer saved to './muril_emotion_final'")
print("✓ Classification head state_dict saved to './muril_emotion_final/classifier_head.pt'")
print("\n" + "="*60)
print("Training complete!")
print("="*60)


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

Calculated pos_weights: tensor([12.1172, 18.1117, 17.6566, 13.2469, 13.1521], device='cuda:0')...
Training samples: 14888
Test samples: 3722

Label distribution in training set:
  admiration: 1135/14888 (7.6%)
  amusement: 779/14888 (5.2%)
  anger: 798/14888 (5.4%)
  annoyance: 1045/14888 (7.0%)
  approval: 1052/14888 (7.1%)


pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/953M [00:00<?, ?B/s]


Model initialized with 237577756 parameters

DIAGNOSTIC: Initial Model Predictions (Before Training)

Logits range: [-0.053, 0.053]
Probabilities range: [0.487, 0.513]
Mean probability: 0.500
Predictions > 0.5: 104 out of 224
✓ Encoder frozen - only classification head will be trained


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

Without Freezing

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModel,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)
from sklearn.metrics import f1_score, accuracy_score, hamming_loss
import torch.nn as nn

# ============================================================
# 0. BASIC CONFIG
# ============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

MODEL_NAME = "google/muril-base-cased"
NUM_LABELS = len(emotion_columns)
MAX_LENGTH = 128

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ============================================================
# 1. DATASET WRAPPER
# ============================================================

class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, df, text_col, label_cols, tokenizer, max_length):
        self.texts = df[text_col].astype(str).tolist()
        self.labels = df[label_cols].astype(np.float32).values
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(labels, dtype=torch.float),
        }

train_dataset = EmotionDataset(
    df=train_combined,
    text_col="text",
    label_cols=emotion_columns,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH,
)

test_dataset = EmotionDataset(
    df=test_combined,
    text_col="text",
    label_cols=emotion_columns,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH,
)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Label stats
train_labels_np = train_combined[emotion_columns].astype(np.float32).values
label_freq = train_labels_np.sum(axis=0)
print("\nLabel frequencies (train):")
for e, c in zip(emotion_columns, label_freq):
    print(f"  {e:15s}: {int(c)} ({c/len(train_dataset)*100:.2f}%)")

# ============================================================
# 2. CLASS IMBALANCE HANDLING (pos_weight)
# ============================================================

num_samples = len(train_dataset)
num_positives = label_freq
num_negatives = num_samples - num_positives

eps = 1e-6
raw_pos_weights = num_negatives / (num_positives + eps)

max_pos_weight = 10.0
pos_weights = np.clip(raw_pos_weights, 1.0, max_pos_weight)
pos_weights = torch.tensor(pos_weights, dtype=torch.float, device=device)

print("\npos_weight per label (clipped):")
for e, w in zip(emotion_columns, pos_weights.cpu().numpy()):
    print(f"  {e:15s}: {w:.2f}")

# ============================================================
# 3. MODEL: MURIL + CLASSIFIER HEAD
# ============================================================

class MURILForMultiLabel(nn.Module):
    def __init__(self, model_name, num_labels, pos_weights, dropout=0.3):
        super().__init__()
        self.num_labels = num_labels
        self.pos_weights = pos_weights

        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size

        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_size, num_labels)

        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        pooled = outputs.last_hidden_state[:, 0, :]
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)

        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.pos_weights)
            loss = loss_fct(logits, labels)

        return {"loss": loss, "logits": logits}

model = MURILForMultiLabel(
    model_name=MODEL_NAME,
    num_labels=NUM_LABELS,
    pos_weights=pos_weights,
    dropout=0.3,
).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nModel initialized: {total_params:,} params ({trainable_params:,} trainable)")

# ============================================================
# 4. METRICS
# ============================================================

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1.0 / (1.0 + np.exp(-logits))  # sigmoid
    preds = (probs > 0.5).astype(int)

    f1_micro = f1_score(labels, preds, average="micro", zero_division=0)
    f1_macro = f1_score(labels, preds, average="macro", zero_division=0)
    f1_weighted = f1_score(labels, preds, average="weighted", zero_division=0)
    hamming = hamming_loss(labels, preds)
    exact_match = accuracy_score(labels, preds)

    avg_labels_per_sample = labels.sum(axis=1).mean()
    avg_preds_per_sample = preds.sum(axis=1).mean()

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
        "hamming_loss": hamming,
        "exact_match": exact_match,
        "avg_labels_per_sample": avg_labels_per_sample,
        "avg_preds_per_sample": avg_preds_per_sample,
    }

# ============================================================
# 5. TRAINING ARGS (use eval_strategy to match your version)
# ============================================================

training_args = TrainingArguments(
    output_dir="./muril_emotion_sota",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    max_grad_norm=1.0,
    logging_dir="./logs_muril_emotion",
    logging_steps=50,
    eval_strategy="steps",      # IMPORTANT: your transformers uses eval_strategy
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    report_to="none",
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# ============================================================
# 6. TRAIN
# ============================================================

print("\n===============================")
print("TRAINING MURIL (FULL FINETUNE)")
print("===============================\n")

train_result = trainer.train()
trainer.save_model("./muril_emotion_sota/best_model")

print("\nBest checkpoint metrics:")
best_metrics = trainer.evaluate()
for k, v in best_metrics.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")
    else:
        print(f"  {k}: {v}")

# ============================================================
# 7. FINAL EVAL WITH THRESHOLD SWEEP
# ============================================================

print("\n========================================")
print("FINAL EVALUATION WITH THRESHOLD SWEEP")
print("========================================\n")

model.eval()
all_logits = []
all_labels = []

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False
)

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        all_logits.append(outputs["logits"].cpu())
        all_labels.append(labels)

all_logits = torch.cat(all_logits, dim=0)
all_labels = torch.cat(all_labels, dim=0)

all_probs = torch.sigmoid(all_logits).numpy()
all_labels_np = all_labels.numpy()

best_macro = -1.0
best_threshold = 0.5

print("Threshold sweep (global):")
for thr in [0.3, 0.4, 0.5, 0.6, 0.7]:
    preds = (all_probs > thr).astype(int)
    f1_macro = f1_score(all_labels_np, preds, average="macro", zero_division=0)
    f1_micro = f1_score(all_labels_np, preds, average="micro", zero_division=0)
    avg_preds = preds.sum(axis=1).mean()
    print(
        f"  thr={thr:.1f} -> "
        f"F1_macro={f1_macro:.4f}, F1_micro={f1_micro:.4f}, "
        f"avg preds/sample={avg_preds:.2f}"
    )
    if f1_macro > best_macro:
        best_macro = f1_macro
        best_threshold = thr

print(f"\nChosen threshold: {best_threshold:.2f} (best macro F1 = {best_macro:.4f})")

final_preds = (all_probs > best_threshold).astype(int)

print("\nFinal metrics with chosen threshold:")
print(
    f"  F1 Macro:     "
    f"{f1_score(all_labels_np, final_preds, average='macro', zero_division=0):.4f}"
)
print(
    f"  F1 Micro:     "
    f"{f1_score(all_labels_np, final_preds, average='micro', zero_division=0):.4f}"
)
print(
    f"  F1 Weighted:  "
    f"{f1_score(all_labels_np, final_preds, average='weighted', zero_division=0):.4f}"
)
print(f"  Hamming Loss: {hamming_loss(all_labels_np, final_preds):.4f}")
print(f"  Exact Match:  {accuracy_score(all_labels_np, final_preds):.4f}")

print("\n===============================")
print("PER-EMOTION DIAGNOSTICS")
print("===============================\n")

for i, emotion in enumerate(emotion_columns):
    f1_e = f1_score(all_labels_np[:, i], final_preds[:, i], zero_division=0)
    support = int(all_labels_np[:, i].sum())
    avg_prob = all_probs[:, i].mean()
    print(
        f"{emotion:15s} | F1: {f1_e:.4f} | "
        f"support: {support:4d} | avg_prob: {avg_prob:.3f}"
    )

# ============================================================
# 8. SAVE ENCODER + TOKENIZER + HEAD
# ============================================================

save_dir = "./muril_emotion_sota/final_export"
model.encoder.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
torch.save(model.classifier.state_dict(), f"{save_dir}/classifier_head.pt")

print(f"\nSaved encoder + tokenizer + classifier head in '{save_dir}'")


Using device: cuda
Training samples: 14897
Test samples: 3725

Label frequencies (train):
  admiration     : 1119 (7.51%)
  amusement      : 779 (5.23%)
  anger          : 758 (5.09%)
  annoyance      : 1031 (6.92%)
  approval       : 1072 (7.20%)
  caring         : 775 (5.20%)
  confusion      : 767 (5.15%)
  curiosity      : 868 (5.83%)
  desire         : 668 (4.48%)
  disappointment : 1028 (6.90%)
  disapproval    : 824 (5.53%)
  disgust        : 738 (4.95%)
  embarrassment  : 670 (4.50%)
  excitement     : 745 (5.00%)
  fear           : 710 (4.77%)
  gratitude      : 931 (6.25%)
  grief          : 563 (3.78%)
  joy            : 867 (5.82%)
  love           : 777 (5.22%)
  nervousness    : 686 (4.60%)
  optimism       : 955 (6.41%)
  pride          : 578 (3.88%)
  realization    : 908 (6.10%)
  relief         : 599 (4.02%)
  remorse        : 642 (4.31%)
  sadness        : 1064 (7.14%)
  surprise       : 699 (4.69%)
  neutral        : 575 (3.86%)

pos_weight per label (clipped):
  ad

Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,F1 Weighted,Hamming Loss,Exact Match,Avg Labels Per Sample,Avg Preds Per Sample,Runtime,Samples Per Second,Steps Per Second
200,0.997400,0.997319,0.101350,0.032694,0.040173,0.064439,0.005369,1.498255,0.509530,5.994300,621.423000,19.519000
400,0.978000,0.967059,0.226364,0.143578,0.160070,0.104660,0.021745,1.498255,2.289664,6.045900,616.117000,19.352000
600,0.946300,0.942223,0.260217,0.213539,0.227455,0.131381,0.009128,1.498255,3.474362,5.942000,626.895000,19.690000
800,0.925400,0.918990,0.295105,0.259081,0.269337,0.114324,0.014228,1.498255,3.042953,5.927100,628.472000,19.740000
1000,0.905400,0.899415,0.306131,0.293282,0.298135,0.114046,0.012617,1.498255,3.103893,6.082500,612.409000,19.235000
1200,0.880800,0.883122,0.312807,0.311083,0.311763,0.122081,0.014228,1.498255,3.475973,6.052000,615.494000,19.332000
1400,0.860100,0.868633,0.336108,0.328288,0.329828,0.104765,0.023893,1.498255,2.920268,5.936800,627.444000,19.708000
1600,0.842900,0.859172,0.339095,0.332735,0.333938,0.107785,0.023356,1.498255,3.068188,6.029000,617.844000,19.406000
1800,0.846900,0.850843,0.341683,0.338826,0.339987,0.105331,0.032483,1.498255,2.981745,6.175300,603.208000,18.946000
2000,0.836500,0.846818,0.340470,0.339042,0.339808,0.109578,0.026846,1.498255,3.153826,6.067100,613.963000,19.284000



Best checkpoint metrics:


  eval_loss: 0.8433
  eval_f1_micro: 0.3414
  eval_f1_macro: 0.3419
  eval_f1_weighted: 0.3420
  eval_hamming_loss: 0.1099
  eval_exact_match: 0.0287
  eval_avg_labels_per_sample: 1.4983
  eval_avg_preds_per_sample: 3.1753
  eval_runtime: 6.2643
  eval_samples_per_second: 594.6420
  eval_steps_per_second: 18.6770
  epoch: 5.0000

FINAL EVALUATION WITH THRESHOLD SWEEP

Threshold sweep (global):
  thr=0.3 -> F1_macro=0.1094, F1_micro=0.1089, avg preds/sample=25.72
  thr=0.4 -> F1_macro=0.2199, F1_micro=0.2019, avg preds/sample=10.71
  thr=0.5 -> F1_macro=0.3419, F1_micro=0.3414, avg preds/sample=3.18
  thr=0.6 -> F1_macro=0.2210, F1_micro=0.2994, avg preds/sample=0.67
  thr=0.7 -> F1_macro=0.0587, F1_micro=0.0954, avg preds/sample=0.10

Chosen threshold: 0.50 (best macro F1 = 0.3419)

Final metrics with chosen threshold:
  F1 Macro:     0.3419
  F1 Micro:     0.3414
  F1 Weighted:  0.3420
  Hamming Loss: 0.1099
  Exact Match:  0.0287

PER-EMOTION DIAGNOSTICS

admiration      | F1: 0.3639

### MURIL + XGBoost

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from xgboost import XGBClassifier # Removed 'callback' as it's not needed with direct args
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, accuracy_score, hamming_loss, classification_report
import pickle
from tqdm import tqdm
import os

# -------------------------------------------------------------------
# CONFIG / PATHS
# -------------------------------------------------------------------
MODEL_PATH = './muril_emotion_final'  # folder you saved: contains tokenizer + base model
CLASSIFIER_HEAD_PATH = os.path.join(MODEL_PATH, 'classifier_head.pt')  # optional
EMBED_MAX_LENGTH = 128
BATCH_SIZE_FE = 32

# Ensure these variables are available in this scope:
# - train_combined, test_combined, emotion_columns
# (They were defined in your training script. If not, load them here.)

NUM_LABELS = len(emotion_columns)

# ============================================================
# 1. LOAD SAVED TOKENIZER + FEATURE EXTRACTOR (MURIL)
# ============================================================
print("Loading tokenizer from original model name and MURIL feature extractor from:", MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) # Load tokenizer from original name

# Load encoder-only (AutoModel) from the saved folder
feature_extractor = AutoModel.from_pretrained(MODEL_PATH)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor.to(device)
feature_extractor.eval()
print(f"✓ Feature extractor loaded on {device} (hidden_size={feature_extractor.config.hidden_size})")


# ============================================================
# 2. EXTRACT FEATURES FROM TEXT
# ============================================================

def extract_features_batch(texts, batch_size=32):
    """
    Extract MURIL embeddings for a list of texts
    Returns: numpy array of shape (n_samples, 768)
    """
    all_features = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Extracting features"):
            batch_texts = texts[i:i+batch_size]

            # Tokenize
            encoded = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=128,
                return_tensors='pt'
            )

            # Move to device
            input_ids = encoded['input_ids'].to(device)
            attention_mask = encoded['attention_mask'].to(device)

            # Get embeddings
            outputs = feature_extractor(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            # Use [CLS] token embedding (first token)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_features.append(cls_embeddings)

    return np.vstack(all_features)

# Extract features for train and test sets
print("\n" + "="*60)
print("EXTRACTING FEATURES")
print("="*60)

train_texts = train_combined['text'].tolist()
train_labels = train_combined[emotion_columns].values.astype(np.float32) # Ensure labels are float32

X_train = extract_features_batch(train_texts, BATCH_SIZE_FE)

test_texts = test_combined['text'].tolist()
test_labels = test_combined[emotion_columns].values.astype(np.float32)   # Ensure labels are float32

X_test = extract_features_batch(test_texts, BATCH_SIZE_FE)

print(f"\n✓ Training features shape: {X_train.shape}")
print(f"✓ Test features shape: {X_test.shape}")

# ============================================================
# 3. XGBOOST HYPERPARAMETERS - OPTIMIZED FOR MULTI-LABEL
# ============================================================

"""
XGBoost Hyperparameter Choices:

1. max_depth=6: Moderate depth to capture emotion patterns without overfitting
2. learning_rate=0.1: Standard rate, balanced convergence
3. n_estimators=200: Enough trees for stable predictions
4. min_child_weight=3: Regularization to prevent overfitting on small classes
5. gamma=0.1: Minimum loss reduction for split (regularization)
6. subsample=0.8: Use 80% of data per tree (prevents overfitting)
7. colsample_bytree=0.8: Use 80% of features per tree (regularization)
8. scale_pos_weight: Automatically computed for each emotion to handle imbalance
9. objective='binary:logistic': Binary classification for each label
10. eval_metric='logloss': Optimize log loss
11. tree_method='hist': Fast histogram-based algorithm
12. random_state=42: Reproducibility
"""

# Calculate scale_pos_weight for each emotion (handles imbalance)
scale_pos_weights = []
for i in range(len(emotion_columns)):
    neg_count = (train_labels[:, i] == 0).sum()
    pos_count = (train_labels[:, i] == 1).sum()
    if pos_count > 0:
        scale_pos_weights.append(neg_count / pos_count)
    else:
        scale_pos_weights.append(1.0)

print("\n" + "="*60)
print("TRAINING XGBOOST CLASSIFIERS")
print("="*60)

# ============================================================
# 4. TRAIN ONE XGBOOST CLASSIFIER PER EMOTION
# ============================================================

# We'll train separate XGBoost models for each emotion
# This gives us more control over per-emotion hyperparameters
xgb_models = {}

for idx, emotion in enumerate(tqdm(emotion_columns, desc="Training XGBoost models")):

    xgb_classifier = XGBClassifier(
        max_depth=6,
        learning_rate=0.1,
        n_estimators=200,
        min_child_weight=3,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weights[idx],  # Handle class imbalance
        objective='binary:logistic',
        eval_metric='logloss',
        tree_method='hist',
        random_state=42,
        early_stopping_rounds=20,
        n_jobs=-1  # Use all CPU cores
    )

    # Train on this emotion's labels
    y_train_emotion = train_labels[:, idx]

    # Early stopping with validation set (10% of train)
    val_size = int(0.1 * len(X_train))
    X_train_split = X_train[:-val_size]
    y_train_split = y_train_emotion[:-val_size]
    X_val_split = X_train[-val_size:]
    y_val_split = y_train_emotion[-val_size:]

    # Early stopping with validation set
    xgb_classifier.fit(
        X_train_split,
        y_train_split,
        eval_set=[(X_val_split, y_val_split)],
        verbose=False # Set to True for verbose output
    )

    xgb_models[emotion] = xgb_classifier

print(f"\n✓ Trained {len(xgb_models)} XGBoost classifiers")

# ============================================================
# 5. MAKE PREDICTIONS
# ============================================================

print("\n" + "="*60)
print("MAKING PREDICTIONS")
print("="*60)

# Predict for each emotion
y_pred_proba = np.zeros((len(X_test), len(emotion_columns)))
y_pred = np.zeros((len(X_test), len(emotion_columns)))

for idx, emotion in enumerate(emotion_columns):
    y_pred_proba[:, idx] = xgb_models[emotion].predict_proba(X_test)[:, 1]
    y_pred[:, idx] = (y_pred_proba[:, idx] > 0.5).astype(int)

# ============================================================
# 6. EVALUATION
# ============================================================

print("\n" + "="*60)
print("EVALUATION METRICS")
print("="*60)

# Overall metrics
f1_micro = f1_score(test_labels, y_pred, average='micro', zero_division=0)
f1_macro = f1_score(test_labels, y_pred, average='macro', zero_division=0)
f1_weighted = f1_score(test_labels, y_pred, average='weighted', zero_division=0)
hamming = hamming_loss(test_labels, y_pred)
exact_match = accuracy_score(test_labels, y_pred)

print(f"\nOverall Performance:")
print(f"  F1 Micro:     {f1_micro:.4f}")
print(f"  F1 Macro:     {f1_macro:.4f}")
print(f"  F1 Weighted:  {f1_weighted:.4f}")
print(f"  Hamming Loss: {hamming:.4f}")
print(f"  Exact Match:  {exact_match:.4f}")

# Per-emotion performance
print("\n" + "="*60)
print("PER-EMOTION PERFORMANCE")
print("="*60 + "\n")

emotion_scores = []
for idx, emotion in enumerate(emotion_columns):
    f1 = f1_score(test_labels[:, idx], y_pred[:, idx], zero_division=0)
    support = test_labels[:, idx].sum()
    emotion_scores.append({
        'emotion': emotion,
        'f1': f1,
        'support': int(support)
    })
    print(f"{emotion:15s} - F1: {f1:.4f} (support: {int(support)})")

# ============================================================
# 7. FEATURE IMPORTANCE ANALYSIS
# ============================================================

print("\n" + "="*60)
print("TOP 3 EMOTIONS BY FEATURE IMPORTANCE")
print("="*60 + "\n")

# Get top emotions by average feature importance
for emotion in emotion_columns[:5]:  # Show top 5 for brevity
    importance = xgb_models[emotion].feature_importances_
    avg_importance = importance.mean()
    print(f"{emotion:15s} - Avg Importance: {avg_importance:.4f}")

# ============================================================
# 8. SAVE MODELS
# ============================================================

print("\n" + "="*60)
print("SAVING MODELS")
print("="*60)

# Save XGBoost models
with open('./xgboost_emotion_models.pkl', 'wb') as f:
    pickle.dump(xgb_models, f)

# Save feature extractor path for later use
with open('./feature_extractor_path.txt', 'w') as f:
    f.write(MODEL_PATH)

print("\n✓ XGBoost models saved to './xgboost_emotion_models.pkl'")
print("✓ Feature extractor path saved")

# ============================================================
# 9. INFERENCE FUNCTION FOR MALAYALAM TEXT
# ============================================================

def predict_emotions_malayalam(text, threshold=0.5):
    """
    Predict emotions for Malayalam text using MURIL + XGBoost pipeline

    Args:
        text: str or list of str (Malayalam text)
        threshold: float (probability threshold for positive label)s

    Returns:
        dict with emotion predictions and probabilities
    """
    # Handle single text or list
    if isinstance(text, str):
        texts = [text]
        single_input = True
    else:
        texts = text
        single_input = False

    # Extract features
    features = extract_features_batch(texts, batch_size=32)

    # Predict with XGBoost
    predictions = {}
    probabilities = {}

    for idx, emotion in enumerate(emotion_columns):
        probs = xgb_models[emotion].predict_proba(features)[:, 1]
        preds = (probs > threshold).astype(int)

        predictions[emotion] = preds
        probabilities[emotion] = probs

    # Format output
    if single_input:
        result = {
            'emotions': [emotion for emotion in emotion_columns
                        if predictions[emotion][0] == 1],
            'probabilities': {emotion: float(probabilities[emotion][0])
                            for emotion in emotion_columns}
        }
    else:
        result = {
            'emotions': [{emotion for emotion in emotion_columns
                         if predictions[emotion][i] == 1}
                        for i in range(len(texts))],
            'probabilities': [{emotion: float(probabilities[emotion][i])
                             for emotion in emotion_columns}
                            for i in range(len(texts))]
        }

    return result

# Test the inference function
print("\n" + "="*60)
print("TESTING INFERENCE FUNCTION")
print("="*60 + "\n")

test_text = test_texts[0]
result = predict_emotions_malayalam(test_text)
print(f"Text: {test_text[:100]}...")
print(f"Predicted emotions: {result['emotions']}")
print(f"Top 3 probabilities:")
sorted_probs = sorted(result['probabilities'].items(),
                     key=lambda x: x[1], reverse=True)[:3]
for emotion, prob in sorted_probs:
    print(f"  {emotion}: {prob:.4f}")

print("\n" + "="*60)
print("MURIL + XGBoost Pipeline Complete!")
print("Ready for Malayalam emotion inference")
print("="*60)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared.")
else:
    print("No GPU detected, skipping cache clear.")

### IndicBERT

In [ ]:
!huggingface-cli login

In [ ]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModel,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import f1_score, accuracy_score, hamming_loss
import torch.nn as nn
import matplotlib.pyplot as plt

# ============================================================
# HYPERPARAMETER CHOICES - DETAILED EXPLANATION
# ============================================================
"""
1. MODEL CHOICE: ai4bharat/indic-bert
   - Pre-trained on 12 major Indian languages including Malayalam
   - Trained on large-scale Indian language corpora (IndicCorp)
   - Better suited for Indian language understanding than multilingual models

2. LEARNING RATE: 2e-5 (classification head), 1e-5 (encoder layers)
   - Lower LR for encoder to preserve cross-lingual representations
   - Higher LR for head since it's trained from scratch

3. BATCH SIZE: 16 (with gradient accumulation steps = 2, effective = 32)
   - Balanced for GPU memory and stable gradients
   - Effective batch size 32 is optimal for transformer fine-tuning

4. EPOCHS: 5-8
   - Start with 5, use early stopping to prevent overfitting
   - Cross-lingual transfer needs careful regularization

5. WARMUP: 10% of training steps
   - Gradual learning rate increase prevents destabilizing pre-trained weights

6. WEIGHT DECAY: 0.01
   - Regularization to maintain generalization to Malayalam

7. DROPOUT: 0.3 (classification head)
   - Higher dropout for regularization during cross-lingual transfer

8. GRADIENT CLIPPING: 1.0
   - Prevents exploding gradients with multi-label BCE loss

9. FREEZING STRATEGY: Gradual unfreezing
   - Phase 1: Freeze encoder, train classification head (3 epochs)
   - Phase 2: Unfreeze top 4 encoder layers (3 epochs)
   - Preserves cross-lingual knowledge while adapting to emotions
"""

"""
IndicBERT Fine-tuning for Multi-Label Emotion Classification
Optimized for Indian languages with proper BCE loss and model architecture
"""

# ============================================================
# 1. SETUP AND CONFIGURATION
# ============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

MODEL_NAME = "ai4bharat/indic-bert"
NUM_LABELS = 28
MAX_LENGTH = 128

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ============================================================
# 2. PREPARE DATASETS WITH PROPER FORMATTING
# ============================================================

class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(label)  # Float for BCE loss
        }

# Prepare data
train_texts = train_combined['text'].tolist()
train_labels = train_combined[emotion_columns].values.astype(np.float32)

test_texts = test_combined['text'].tolist()
test_labels = test_combined[emotion_columns].values.astype(np.float32)

train_dataset = EmotionDataset(train_texts, train_labels, tokenizer, MAX_LENGTH)
test_dataset = EmotionDataset(test_texts, test_labels, tokenizer, MAX_LENGTH)

# Calculate positive weights for class imbalance
num_positives = train_labels.sum(axis=0)
num_samples = len(train_dataset)
num_negatives = num_samples - num_positives
# Calculate ratio: negatives / positives
pos_weights = torch.tensor(num_negatives / num_positives, dtype=torch.float32)
pos_weights = pos_weights.to(device)  # Move to GPU
print(f"Calculated pos_weights (first 5): {pos_weights[:5].cpu().numpy()}")

print(f"\nTraining samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Check label distribution
print("\nLabel distribution in training set:")
for idx, emotion in enumerate(emotion_columns[:5]):  # Show first 5
    pos_count = (train_labels[:, idx] == 1).sum()
    print(f"  {emotion}: {pos_count}/{len(train_labels)} ({pos_count/len(train_labels)*100:.1f}%)")

# ============================================================
# 3. MODEL ARCHITECTURE FOR INDICBERT
# ============================================================

class IndicBERTForMultiLabelClassification(nn.Module):
    def __init__(self, model_name, num_labels, pos_weights=None, dropout=0.3):
        super().__init__()
        self.num_labels = num_labels
        self.pos_weights = pos_weights

        # Load IndicBERT base model
        self.indicbert = AutoModel.from_pretrained(model_name)

        # Classification head with dropout
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.indicbert.config.hidden_size, num_labels)

        # Initialize classifier weights properly
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(self, input_ids, attention_mask, labels=None):
        # Get IndicBERT outputs
        outputs = self.indicbert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Use [CLS] token representation (first token)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            # Binary Cross Entropy with Logits Loss with positive weights
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.pos_weights)
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))

        return {'loss': loss, 'logits': logits}

# Initialize model
model = IndicBERTForMultiLabelClassification(
    MODEL_NAME,
    NUM_LABELS,
    pos_weights=pos_weights,
    dropout=0.3
)
model.to(device)

print(f"\nModel initialized with {sum(p.numel() for p in model.parameters()):,} parameters")

# ============================================================
# 4. DIAGNOSTIC: Check Initial Predictions
# ============================================================

print("\n" + "="*60)
print("DIAGNOSTIC: Initial Model Predictions (Before Training)")
print("="*60 + "\n")

model.eval()
with torch.no_grad():
    sample_batch = next(iter(torch.utils.data.DataLoader(train_dataset, batch_size=8)))
    sample_input_ids = sample_batch['input_ids'].to(device)
    sample_attention_mask = sample_batch['attention_mask'].to(device)

    outputs = model(sample_input_ids, sample_attention_mask)
    sample_logits = outputs['logits']
    sample_probs = torch.sigmoid(sample_logits)

    print(f"Logits range: [{sample_logits.min():.3f}, {sample_logits.max():.3f}]")
    print(f"Probabilities range: [{sample_probs.min():.3f}, {sample_probs.max():.3f}]")
    print(f"Mean probability: {sample_probs.mean():.3f}")
    print(f"Predictions > 0.5: {(sample_probs > 0.5).sum().item()} out of {sample_probs.numel()}")

# ============================================================
# 5. FREEZING STRATEGY
# ============================================================

def freeze_encoder(model):
    """Freeze IndicBERT encoder, keep classification head trainable"""
    for param in model.indicbert.parameters():
        param.requires_grad = False
    # Ensure classifier is trainable
    for param in model.classifier.parameters():
        param.requires_grad = True
    print("✓ Encoder frozen - only classification head will be trained")

def unfreeze_top_layers(model, num_layers=4):
    """Unfreeze top N encoder layers in an ALBERT-like model."""
    # Collect all individual AlbertLayer instances
    all_albert_layers = []
    for group in model.indicbert.encoder.albert_layer_groups:
        for albert_layer in group.albert_layers:
            all_albert_layers.append(albert_layer)

    total_layers = len(all_albert_layers)

    if num_layers > total_layers:
        print(f"Warning: Requested to unfreeze {num_layers} layers, but model only has {total_layers} layers. Unfreezing all layers.")
        layers_to_unfreeze = all_albert_layers
    else:
        layers_to_unfreeze = all_albert_layers[total_layers - num_layers:]

    for layer in layers_to_unfreeze:
        for param in layer.parameters():
            param.requires_grad = True
    print(f"✓ Top {len(layers_to_unfreeze)} encoder layers unfrozen")

freeze_encoder(model)

# ============================================================
# 6. IMPROVED METRICS WITH DEBUGGING
# ============================================================

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert logits to probabilities
    probs = 1 / (1 + np.exp(-logits))  # Sigmoid
    predictions = (probs > 0.5).astype(int)

    # Debug: Check prediction distribution
    num_positives = predictions.sum()
    total_predictions = predictions.size

    # Multi-label metrics
    f1_micro = f1_score(labels, predictions, average='micro', zero_division=0)
    f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)
    f1_weighted = f1_score(labels, predictions, average='weighted', zero_division=0)
    hamming = hamming_loss(labels, predictions)
    exact_match = accuracy_score(labels, predictions)

    # Additional diagnostic metrics
    avg_labels_per_sample = labels.sum(axis=1).mean()
    avg_predictions_per_sample = predictions.sum(axis=1).mean()

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'hamming_loss': hamming,
        'exact_match': exact_match,
        'num_positive_preds': num_positives,
        'avg_labels_per_sample': avg_labels_per_sample,
        'avg_preds_per_sample': avg_predictions_per_sample
    }

# ============================================================
# 7. PHASE 1 TRAINING - CLASSIFICATION HEAD ONLY
# ============================================================

training_args_phase1 = TrainingArguments(
    output_dir='./indicbert_emotion_phase1',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,  # Higher LR for classification head
    weight_decay=0.01,
    warmup_ratio=0.15,
    logging_dir='./logs_phase1',
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='f1_macro',
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    report_to="none",
    seed=42,
    label_smoothing_factor=0.0  # No label smoothing for multi-label
)

print("\n" + "="*60)
print("PHASE 1: Training Classification Head (Encoder Frozen)")
print("="*60 + "\n")

trainer_phase1 = Trainer(
    model=model,
    args=training_args_phase1,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

trainer_phase1.train()

# Evaluate Phase 1
phase1_results = trainer_phase1.evaluate()
print("\nPhase 1 Results:")
for key, value in phase1_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# ============================================================
# 8. POST-PHASE 1 DIAGNOSTIC
# ============================================================

print("\n" + "="*60)
print("POST-PHASE 1 DIAGNOSTIC")
print("="*60 + "\n")

model.eval()
with torch.no_grad():
    sample_batch = next(iter(torch.utils.data.DataLoader(test_dataset, batch_size=8)))
    sample_input_ids = sample_batch['input_ids'].to(device)
    sample_attention_mask = sample_batch['attention_mask'].to(device)
    sample_labels = sample_batch['labels']

    outputs = model(sample_input_ids, sample_attention_mask)
    sample_logits = outputs['logits']
    sample_probs = torch.sigmoid(sample_logits)

    print(f"Logits range: [{sample_logits.min():.3f}, {sample_logits.max():.3f}]")
    print(f"Probabilities range: [{sample_probs.min():.3f}, {sample_probs.max():.3f}]")
    print(f"Mean probability: {sample_probs.mean():.3f}")
    print(f"Predictions > 0.5: {(sample_probs > 0.5).sum().item()} out of {sample_probs.numel()}")
    print(f"Actual positive labels: {sample_labels.sum().item()}")

# ============================================================
# 9. PHASE 2 - UNFREEZE TOP LAYERS
# ============================================================

# Only continue to Phase 2 if model is learning
if phase1_results['eval_f1_macro'] > 0.01:
    print("\n✓ Model is learning! Proceeding to Phase 2...")

    print("\n" + "="*60)
    print("PHASE 2: Fine-tuning Top Encoder Layers")
    print("="*60 + "\n")

    unfreeze_top_layers(model, num_layers=4)

    training_args_phase2 = TrainingArguments(
        output_dir='./indicbert_emotion_phase2',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=2,
        learning_rate=2e-5,  # Lower LR for fine-tuning
        weight_decay=0.01,
        warmup_ratio=0.15,
        max_grad_norm=1.0,
        logging_dir='./logs_phase2',
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=200,
        save_strategy="steps",
        save_steps=200,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        fp16=torch.cuda.is_available(),
        dataloader_num_workers=2,
        report_to="none",
        seed=42
    )

    trainer_phase2 = Trainer(
        model=model,
        args=training_args_phase2,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

    trainer_phase2.train()
    final_results = trainer_phase2.evaluate()

else:
    print("\n⚠ WARNING: Model not learning in Phase 1!")
    print("Skipping Phase 2. Need to debug further.")
    final_results = phase1_results

# ============================================================
# 10. FINAL EVALUATION WITH ADAPTIVE THRESHOLD
# ============================================================

print("\n" + "="*60)
print("FINAL EVALUATION WITH ADAPTIVE THRESHOLD")
print("="*60 + "\n")

# Collect all predictions
model.eval()
all_logits = []
all_labels = []

with torch.no_grad():
    for batch in torch.utils.data.DataLoader(test_dataset, batch_size=32):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask)
        all_logits.append(outputs['logits'].cpu())
        all_labels.append(batch['labels'])

all_logits = torch.cat(all_logits, dim=0)
all_labels = torch.cat(all_labels, dim=0)
all_probs = torch.sigmoid(all_logits).numpy()
all_labels = all_labels.numpy()

# Test different thresholds
print("Testing different prediction thresholds:")
for threshold in [0.3, 0.4, 0.5, 0.6, 0.7]:
    preds = (all_probs > threshold).astype(int)
    f1_macro = f1_score(all_labels, preds, average='macro', zero_division=0)
    f1_micro = f1_score(all_labels, preds, average='micro', zero_division=0)
    avg_preds = preds.sum() / len(preds)
    print(f"  Threshold {threshold:.1f} - F1 Macro: {f1_macro:.4f}, F1 Micro: {f1_micro:.4f}, Avg preds/sample: {avg_preds:.2f}")

# Use best threshold
best_threshold = 0.5
pred_labels = (all_probs > best_threshold).astype(int)

print(f"\nUsing threshold: {best_threshold}")
print("\nFinal Results:")
print(f"  F1 Macro:     {f1_score(all_labels, pred_labels, average='macro', zero_division=0):.4f}")
print(f"  F1 Micro:     {f1_score(all_labels, pred_labels, average='micro', zero_division=0):.4f}")
print(f"  F1 Weighted:  {f1_score(all_labels, pred_labels, average='weighted', zero_division=0):.4f}")
print(f"  Hamming Loss: {hamming_loss(all_labels, pred_labels):.4f}")
print(f"  Exact Match:  {accuracy_score(all_labels, pred_labels):.4f}")

# Per-emotion performance
print("\n" + "="*60)
print("PER-EMOTION PERFORMANCE")
print("="*60 + "\n")

for idx, emotion in enumerate(emotion_columns):
    f1 = f1_score(all_labels[:, idx], pred_labels[:, idx], zero_division=0)
    support = all_labels[:, idx].sum()
    avg_prob = all_probs[:, idx].mean()
    print(f"{emotion:15s} - F1: {f1:.4f}, Support: {int(support):4d}, Avg Prob: {avg_prob:.3f}")

# ============================================================
# 11. SAVE MODEL
# ============================================================

# Save the IndicBERT feature extractor
model.indicbert.save_pretrained('./indicbert_emotion_final')

# Save the tokenizer
tokenizer.save_pretrained('./indicbert_emotion_final')

# Save the classification head state_dict
torch.save(model.classifier.state_dict(), './indicbert_emotion_final/classifier_head.pt')

print("\n✓ IndicBERT feature extractor and tokenizer saved to './indicbert_emotion_final'")
print("✓ Classification head state_dict saved to './indicbert_emotion_final/classifier_head.pt'")
print("\n" + "="*60)
print("Training complete!")
print("="*60)


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from xgboost import XGBClassifier # Removed 'callback' as it's not needed with direct args
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, accuracy_score, hamming_loss, classification_report
import pickle
from tqdm import tqdm
import os

# -------------------------------------------------------------------
# CONFIG / PATHS
# -------------------------------------------------------------------
MODEL_PATH = './indicbert_emotion_final'  # folder you saved: contains tokenizer + base model
CLASSIFIER_HEAD_PATH = os.path.join(MODEL_PATH, 'classifier_head.pt')  # optional
EMBED_MAX_LENGTH = 128
BATCH_SIZE_FE = 32
MODEL_NAME = "ai4bharat/indic-bert"

# Ensure these variables are available in this scope:
# - train_combined, test_combined, emotion_columns
# (They were defined in your training script. If not, load them here.)

NUM_LABELS = len(emotion_columns)

# ============================================================
# 1. LOAD SAVED TOKENIZER + FEATURE EXTRACTOR (MURIL)
# ============================================================
print("Loading tokenizer from original model name and inidcBERT feature extractor from:", MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) # Load tokenizer from original name

# Load encoder-only (AutoModel) from the saved folder
feature_extractor = AutoModel.from_pretrained(MODEL_PATH)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor.to(device)
feature_extractor.eval()
print(f"✓ Feature extractor loaded on {device} (hidden_size={feature_extractor.config.hidden_size})")


# ============================================================
# 2. EXTRACT FEATURES FROM TEXT
# ============================================================

def extract_features_batch(texts, batch_size=32):
    """
    Extract MURIL embeddings for a list of texts
    Returns: numpy array of shape (n_samples, 768)
    """
    all_features = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Extracting features"):
            batch_texts = texts[i:i+batch_size]

            # Tokenize
            encoded = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=128,
                return_tensors='pt'
            )

            # Move to device
            input_ids = encoded['input_ids'].to(device)
            attention_mask = encoded['attention_mask'].to(device)

            # Get embeddings
            outputs = feature_extractor(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            # Use [CLS] token embedding (first token)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_features.append(cls_embeddings)

    return np.vstack(all_features)

# Extract features for train and test sets
print("\n" + "="*60)
print("EXTRACTING FEATURES")
print("="*60)

train_texts = train_combined['text'].tolist()
train_labels = train_combined[emotion_columns].values.astype(np.float32) # Ensure labels are float32

X_train = extract_features_batch(train_texts, BATCH_SIZE_FE)

test_texts = test_combined['text'].tolist()
test_labels = test_combined[emotion_columns].values.astype(np.float32)   # Ensure labels are float32

X_test = extract_features_batch(test_texts, BATCH_SIZE_FE)

print(f"\n✓ Training features shape: {X_train.shape}")
print(f"✓ Test features shape: {X_test.shape}")

# ============================================================
# 3. XGBOOST HYPERPARAMETERS - OPTIMIZED FOR MULTI-LABEL
# ============================================================

"""
XGBoost Hyperparameter Choices:

1. max_depth=6: Moderate depth to capture emotion patterns without overfitting
2. learning_rate=0.1: Standard rate, balanced convergence
3. n_estimators=200: Enough trees for stable predictions
4. min_child_weight=3: Regularization to prevent overfitting on small classes
5. gamma=0.1: Minimum loss reduction for split (regularization)
6. subsample=0.8: Use 80% of data per tree (prevents overfitting)
7. colsample_bytree=0.8: Use 80% of features per tree (regularization)
8. scale_pos_weight: Automatically computed for each emotion to handle imbalance
9. objective='binary:logistic': Binary classification for each label
10. eval_metric='logloss': Optimize log loss
11. tree_method='hist': Fast histogram-based algorithm
12. random_state=42: Reproducibility
"""

# Calculate scale_pos_weight for each emotion (handles imbalance)
scale_pos_weights = []
for i in range(len(emotion_columns)):
    neg_count = (train_labels[:, i] == 0).sum()
    pos_count = (train_labels[:, i] == 1).sum()
    if pos_count > 0:
        scale_pos_weights.append(neg_count / pos_count)
    else:
        scale_pos_weights.append(1.0)

print("\n" + "="*60)
print("TRAINING XGBOOST CLASSIFIERS")
print("="*60)

# ============================================================
# 4. TRAIN ONE XGBOOST CLASSIFIER PER EMOTION
# ============================================================

# We'll train separate XGBoost models for each emotion
# This gives us more control over per-emotion hyperparameters
xgb_models = {}

for idx, emotion in enumerate(tqdm(emotion_columns, desc="Training XGBoost models")):

    xgb_classifier = XGBClassifier(
        max_depth=6,
        learning_rate=0.1,
        n_estimators=200,
        min_child_weight=3,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weights[idx],  # Handle class imbalance
        objective='binary:logistic',
        eval_metric='logloss',
        tree_method='hist',
        random_state=42,
        early_stopping_rounds=20,
        n_jobs=-1  # Use all CPU cores
    )

    # Train on this emotion's labels
    y_train_emotion = train_labels[:, idx]

    # Early stopping with validation set (10% of train)
    val_size = int(0.1 * len(X_train))
    X_train_split = X_train[:-val_size]
    y_train_split = y_train_emotion[:-val_size]
    X_val_split = X_train[-val_size:]
    y_val_split = y_train_emotion[-val_size:]

    # Early stopping with validation set
    xgb_classifier.fit(
        X_train_split,
        y_train_split,
        eval_set=[(X_val_split, y_val_split)],
        verbose=False # Set to True for verbose output
    )

    xgb_models[emotion] = xgb_classifier

print(f"\n✓ Trained {len(xgb_models)} XGBoost classifiers")

# ============================================================
# 5. MAKE PREDICTIONS
# ============================================================

print("\n" + "="*60)
print("MAKING PREDICTIONS")
print("="*60)

# Predict for each emotion
y_pred_proba = np.zeros((len(X_test), len(emotion_columns)))
y_pred = np.zeros((len(X_test), len(emotion_columns)))

for idx, emotion in enumerate(emotion_columns):
    y_pred_proba[:, idx] = xgb_models[emotion].predict_proba(X_test)[:, 1]
    y_pred[:, idx] = (y_pred_proba[:, idx] > 0.5).astype(int)

# ============================================================
# 6. EVALUATION
# ============================================================

print("\n" + "="*60)
print("EVALUATION METRICS")
print("="*60)

# Overall metrics
f1_micro = f1_score(test_labels, y_pred, average='micro', zero_division=0)
f1_macro = f1_score(test_labels, y_pred, average='macro', zero_division=0)
f1_weighted = f1_score(test_labels, y_pred, average='weighted', zero_division=0)
hamming = hamming_loss(test_labels, y_pred)
exact_match = accuracy_score(test_labels, y_pred)

print(f"\nOverall Performance:")
print(f"  F1 Micro:     {f1_micro:.4f}")
print(f"  F1 Macro:     {f1_macro:.4f}")
print(f"  F1 Weighted:  {f1_weighted:.4f}")
print(f"  Hamming Loss: {hamming:.4f}")
print(f"  Exact Match:  {exact_match:.4f}")

# Per-emotion performance
print("\n" + "="*60)
print("PER-EMOTION PERFORMANCE")
print("="*60 + "\n")

emotion_scores = []
for idx, emotion in enumerate(emotion_columns):
    f1 = f1_score(test_labels[:, idx], y_pred[:, idx], zero_division=0)
    support = test_labels[:, idx].sum()
    emotion_scores.append({
        'emotion': emotion,
        'f1': f1,
        'support': int(support)
    })
    print(f"{emotion:15s} - F1: {f1:.4f} (support: {int(support)})")

# ============================================================
# 7. FEATURE IMPORTANCE ANALYSIS
# ============================================================

print("\n" + "="*60)
print("TOP 3 EMOTIONS BY FEATURE IMPORTANCE")
print("="*60 + "\n")

# Get top emotions by average feature importance
for emotion in emotion_columns[:5]:  # Show top 5 for brevity
    importance = xgb_models[emotion].feature_importances_
    avg_importance = importance.mean()
    print(f"{emotion:15s} - Avg Importance: {avg_importance:.4f}")

# ============================================================
# 8. SAVE MODELS
# ============================================================

print("\n" + "="*60)
print("SAVING MODELS")
print("="*60)

# Save XGBoost models
with open('./xgboost_emotion_models.pkl', 'wb') as f:
    pickle.dump(xgb_models, f)

# Save feature extractor path for later use
with open('./feature_extractor_path.txt', 'w') as f:
    f.write(MODEL_PATH)

print("\n✓ XGBoost models saved to './xgboost_emotion_models.pkl'")
print("✓ Feature extractor path saved")

# ============================================================
# 9. INFERENCE FUNCTION FOR MALAYALAM TEXT
# ============================================================

def predict_emotions_malayalam(text, threshold=0.5):
    """
    Predict emotions for Malayalam text using MURIL + XGBoost pipeline

    Args:
        text: str or list of str (Malayalam text)
        threshold: float (probability threshold for positive label)s

    Returns:
        dict with emotion predictions and probabilities
    """
    # Handle single text or list
    if isinstance(text, str):
        texts = [text]
        single_input = True
    else:
        texts = text
        single_input = False

    # Extract features
    features = extract_features_batch(texts, batch_size=32)

    # Predict with XGBoost
    predictions = {}
    probabilities = {}

    for idx, emotion in enumerate(emotion_columns):
        probs = xgb_models[emotion].predict_proba(features)[:, 1]
        preds = (probs > threshold).astype(int)

        predictions[emotion] = preds
        probabilities[emotion] = probs

    # Format output
    if single_input:
        result = {
            'emotions': [emotion for emotion in emotion_columns
                        if predictions[emotion][0] == 1],
            'probabilities': {emotion: float(probabilities[emotion][0])
                            for emotion in emotion_columns}
        }
    else:
        result = {
            'emotions': [{emotion for emotion in emotion_columns
                         if predictions[emotion][i] == 1}
                        for i in range(len(texts))],
            'probabilities': [{emotion: float(probabilities[emotion][i])
                             for emotion in emotion_columns}
                            for i in range(len(texts))]
        }

    return result

# Test the inference function
print("\n" + "="*60)
print("TESTING INFERENCE FUNCTION")
print("="*60 + "\n")

test_text = test_texts[0]
result = predict_emotions_malayalam(test_text)
print(f"Text: {test_text[:100]}...")
print(f"Predicted emotions: {result['emotions']}")
print(f"Top 3 probabilities:")
sorted_probs = sorted(result['probabilities'].items(),
                     key=lambda x: x[1], reverse=True)[:3]
for emotion, prob in sorted_probs:
    print(f"  {emotion}: {prob:.4f}")

print("\n" + "="*60)
print("MURIL + XGBoost Pipeline Complete!")
print("Ready for Malayalam emotion inference")
print("="*60)

# Flagging Suicidal Language

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModel,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    classification_report,
    confusion_matrix,
    cohen_kappa_score,
    mean_squared_error,
    mean_absolute_error
)
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

## Data Preprocessing

In [ ]:
# ============================================================
# UNDERSTANDING C-SSRS ORDINAL LABELS
# ============================================================
"""
C-SSRS (Columbia Suicide Severity Rating Scale) - ORDINAL CLASSIFICATION

Dataset Distribution:
- Ideation:    171 (34.2%) - Level 2
- Supportive:  108 (21.6%) - Level 0 (lowest severity)
- Indicator:    99 (19.8%) - Level 1
- Behavior:     77 (15.4%) - Level 3
- Attempt:      45 ( 9.0%) - Level 4 (highest severity)

Total: 500 samples (highly imbalanced, especially Attempt class)

ORDINAL SEVERITY SCALE:
0. Supportive   - No risk, providing support to others
1. Indicator    - Risk indicators present but no active ideation
2. Ideation     - Suicidal thoughts present (most common)
3. Behavior     - Preparatory behaviors for suicide
4. Attempt      - Actual suicide attempt (least common)

KEY INSIGHTS FOR ORDINAL MODELING:
- Predicting "Attempt" as "Behavior" is better than predicting as "Supportive"
- We'll use ordinal loss that penalizes distance from true label
- Class weights to handle severe imbalance (Attempt only 9%)
- Use Mean Absolute Error (MAE) as a key metric for ordinal distance
"""

# ============================================================
# 1. DATA LOADING AND EXPLORATION
# ============================================================

print("="*70)
print("LOADING C-SSRS SUICIDALITY DATASET - ORDINAL CLASSIFICATION")
print("="*70 + "\n")

# Load the dataset
try:
    df = pd.read_csv('/content/PHQ9-Malayalam/CSSRS.csv')
    print("✓ Loaded csv")
except:

    print("⚠ Dataset not found. Please ensure the CSV file is in the directory.")
    raise FileNotFoundError("Dataset file not found")

print(f"\nDataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

In [ ]:
# ============================================================
# 2. DATA CLEANING AND PREPROCESSING
# ============================================================

print("\n" + "="*70)
print("DATA CLEANING AND PREPROCESSING")
print("="*70 + "\n")

# Identify text and label columns
text_col = None
label_col = None

for col in df.columns:
    col_lower = col.lower()
    if 'text' in col_lower or 'post' in col_lower or 'content' in col_lower:
        text_col = col
    if 'label' in col_lower or 'class' in col_lower:
        label_col = col

if text_col is None:
    text_col = df.columns[0]
if label_col is None:
    label_col = df.columns[1] if len(df.columns) > 1 else df.columns[-1]

print(f"Using text column: '{text_col}'")
print(f"Using label column: '{label_col}'")

# Rename for consistency
df = df.rename(columns={text_col: 'text', label_col: 'label'})

# Remove rows with missing values
print(f"\nMissing values:")
print(df[['text', 'label']].isnull().sum())

df = df.dropna(subset=['text', 'label'])
print(f"✓ Removed missing values. Shape: {df.shape}")

# Clean text function
def clean_text(text):
    """Clean Reddit post text while preserving emotion indicators"""
    if not isinstance(text, str):
        return ""

    # Remove URLs but keep text structure
    text = re.sub(r'http\S+|www\S+|https\S+', '[URL]', text, flags=re.MULTILINE)

    # Remove Reddit-specific formatting
    text = re.sub(r'\[deleted\]|\[removed\]', '', text)

    # Remove excessive special characters but keep punctuation
    text = re.sub(r'[^\w\s.,!?;:\'\"-]', ' ', text)

    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Filter very short posts
    if len(text) < 10:
        return ""

    return text

df['text'] = df['text'].apply(clean_text)
df = df[df['text'] != ""]
print(f"✓ Text cleaned. Shape: {df.shape}")

In [ ]:
def standardize_label(label):
    """Convert to standard label format"""
    if isinstance(label, str):
        label = label.strip().lower()
        label = label.replace(' ', '').replace('_', '')

        if label in ['a', 'supportive', 'support', 'notrisk']:
            return 'Supportive'
        elif label in ['b', 'indicator', 'ind', 'risk']:
            return 'Indicator'
        elif label in ['c', 'ideation', 'idea', 'thoughts']:
            return 'Ideation'
        elif label in ['d', 'behavior', 'behav', 'preparation']:
            return 'Behavior'
        elif label in ['e', 'attempt', 'att', 'try']:
            return 'Attempt'
    return str(label).strip().title()

df['label'] = df['label'].apply(standardize_label)

# Verify we have the expected distribution
print("\n" + "="*70)
print("LABEL DISTRIBUTION (Should match: Ideation 171, Supportive 108, etc.)")
print("="*70 + "\n")

label_counts = df['label'].value_counts()
print(label_counts)
print(f"\nTotal samples: {len(df)}")
print("\nPercentage distribution:")
for label, count in label_counts.items():
    print(f"  {label:12s}: {count:3d} ({count/len(df)*100:5.1f}%)")

# ORDINAL label mapping (Supportive=0 to Attempt=4)
label_to_id = {
    'Supportive': 0,
    'Indicator': 1,
    'Ideation': 2,
    'Behavior': 3,
    'Attempt': 4
}
id_to_label = {v: k for k, v in label_to_id.items()}

df['label_id'] = df['label'].map(label_to_id)

# Remove any unmapped labels
df = df.dropna(subset=['label_id'])
df['label_id'] = df['label_id'].astype(int)
print(f"\n✓ Labels standardized. Final shape: {df.shape}")

In [ ]:
# ============================================================
# 3. CLASS IMBALANCE ANALYSIS
# ============================================================

print("\n" + "="*70)
print("CLASS IMBALANCE ANALYSIS (Critical for Attempt class with only 45 samples)")
print("="*70 + "\n")

class_counts = df['label_id'].value_counts().sort_index()
for label_id in sorted(label_to_id.values()):
    label_name = id_to_label[label_id]
    count = class_counts.get(label_id, 0)
    percentage = count / len(df) * 100
    print(f"Level {label_id} - {label_name:12s}: {count:3d} samples ({percentage:5.1f}%)")

# Calculate class weights - more aggressive for rare classes
total_samples = len(df)
class_weights = []
for label_id in sorted(label_to_id.values()):
    count = class_counts.get(label_id, 1)
    weight = total_samples / (len(label_to_id) * count)
    # NO smoothing - use raw weights to force minority learning
    weight = weight * 2.0  # Double the weights
    class_weights.append(weight)

class_weights = torch.FloatTensor(class_weights)
print(f"\nClass weights (sqrt-smoothed, amplified for rare classes):")
for label_id, weight in enumerate(class_weights):
    print(f"  {id_to_label[label_id]:12s}: {weight:.3f}")

# Text statistics
df['text_length'] = df['text'].str.len()
df['word_count'] = df['text'].str.split().str.len()

print("\n" + "="*70)
print("TEXT STATISTICS BY SEVERITY LEVEL")
print("="*70 + "\n")

for label_id in sorted(label_to_id.values()):
    label_name = id_to_label[label_id]
    subset = df[df['label_id'] == label_id]
    avg_len = subset['text_length'].mean()
    avg_words = subset['word_count'].mean()
    print(f"{label_name:12s}: Avg length={avg_len:6.1f} chars, Avg words={avg_words:5.1f}")

## Training

In [ ]:


def augment_minority_classes(train_df, label_col='label_id', target_samples=150):
    """Oversample minority classes to balance dataset"""
    from sklearn.utils import resample

    frames = []
    for label_id in sorted(train_df[label_col].unique()):
        subset = train_df[train_df[label_col] == label_id]

        if len(subset) < target_samples:
            # Oversample minority class
            subset_upsampled = resample(
                subset,
                replace=True,
                n_samples=target_samples,
                random_state=42
            )
            frames.append(subset_upsampled)
        else:
            frames.append(subset)

    return pd.concat(frames, ignore_index=True)

# ============================================================
# 4. STRATIFIED K-FOLD FOR ROBUST EVALUATION
# ============================================================

print("\n" + "="*70)
print("CREATING STRATIFIED TRAIN-VALIDATION-TEST SPLIT")
print("="*70 + "\n")

# First, separate test set (20%)
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label_id']
)

# Then split train into train and validation (80-20 of remaining)
train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.2,
    random_state=42,
    stratify=train_val_df['label_id']
)

# Oversample minority classes in training set
print("\nBalancing training set with oversampling...")
print(f"Before: {len(train_df)} samples")
train_df = augment_minority_classes(train_df, label_col='label_id', target_samples=120)
print(f"After: {len(train_df)} samples")
print("New distribution:")
print(train_df['label'].value_counts())

print(f"Training samples:   {len(train_df):3d} ({len(train_df)/len(df)*100:.1f}%)")
print(f"Validation samples: {len(val_df):3d} ({len(val_df)/len(df)*100:.1f}%)")
print(f"Test samples:       {len(test_df):3d} ({len(test_df)/len(df)*100:.1f}%)")

print("\nTraining set distribution:")
print(train_df['label'].value_counts().sort_index())

print("\nValidation set distribution:")
print(val_df['label'].value_counts().sort_index())

print("\nTest set distribution:")
print(test_df['label'].value_counts().sort_index())

In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared.")
else:
    print("No GPU detected, skipping cache clear.")

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 5. DATASET CLASS WITH ORDINAL AWARENESS
# ============================================================

class CSSRSDataset(Dataset):
    """Dataset with optional augmentation for minority classes"""
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
class OrdinalCSSRSClassifier(nn.Module):
    """
    Ordinal classification model using MURIL for cross-lingual transfer.
    Uses ordinal loss to penalize distance from true severity level.
    """
    def __init__(self, model_name, num_classes=5, dropout=0.2):
        super().__init__()
        self.num_classes = num_classes

        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size

        # LESS AGGRESSIVE FREEZING - only freeze embeddings
        for param in self.encoder.embeddings.parameters():
            param.requires_grad = False

        # Improved classification head with batch norm
        self.dropout = nn.Dropout(dropout)
        self.batch_norm = nn.BatchNorm1d(hidden_size)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.batch_norm(pooled_output)
        pooled_output = self.dropout(pooled_output)

        logits = self.classifier(pooled_output)
        return logits


In [ ]:
class FocalLoss(nn.Module):
    """
    Focal Loss to handle severe class imbalance.
    Better than simple class weighting for rare classes.
    """
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.alpha = alpha  # class weights
        self.gamma = gamma  # focusing parameter

    def forward(self, logits, labels):
        ce_loss = nn.functional.cross_entropy(logits, labels, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.alpha is not None:
            alpha_t = self.alpha[labels]
            focal_loss = alpha_t * focal_loss

        return focal_loss.mean()


class OrdinalCrossEntropyLoss(nn.Module):
    """
    Weighted ordinal cross-entropy loss.
    Penalizes predictions based on distance from true label.
    """
    def __init__(self, class_weights=None, ordinal_weight=0.3):
        super().__init__()
        self.class_weights = class_weights
        self.ordinal_weight = ordinal_weight
        self.ce_loss = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, logits, labels):
        # Standard cross-entropy loss with class weights
        ce = self.ce_loss(logits, labels)

        # Ordinal penalty: penalize distance from true label
        preds = torch.argmax(logits, dim=1)
        ordinal_penalty = torch.abs(preds - labels).float().mean()

        # Combined loss
        total_loss = ce + self.ordinal_weight * ordinal_penalty
        return total_loss


class ImprovedOrdinalLoss(nn.Module):
    """Combines Focal Loss with ordinal penalty"""
    def __init__(self, class_weights=None, gamma=2.0, ordinal_weight=0.5):
        super().__init__()
        self.focal_loss = FocalLoss(alpha=class_weights, gamma=gamma)
        self.ordinal_weight = ordinal_weight

    def forward(self, logits, labels):
        # Focal loss for imbalance
        focal = self.focal_loss(logits, labels)

        # Ordinal penalty
        preds = torch.argmax(logits, dim=1)
        ordinal_penalty = torch.abs(preds.float() - labels.float()).mean()

        return focal + self.ordinal_weight * ordinal_penalty

# ============================================================
# 8. TRAINING FUNCTION
# ============================================================

def train_epoch(model, dataloader, optimizer, scheduler, criterion, device, accumulation_steps=2):
    """Train with gradient accumulation for effective larger batch size"""
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    optimizer.zero_grad()
    progress_bar = tqdm(dataloader, desc="Training")

    for batch_idx, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)

        # Normalize loss for accumulation
        loss = loss / accumulation_steps
        loss.backward()

        # Gradient accumulation
        if (batch_idx + 1) % accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        progress_bar.set_postfix({'loss': f'{loss.item() * accumulation_steps:.4f}'})

    avg_loss = total_loss / len(dataloader)
    mae = mean_absolute_error(all_labels, all_preds)
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))

    return avg_loss, accuracy, mae

def evaluate(model, dataloader, criterion, device):
    """Evaluate model"""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)

            total_loss += loss.item()
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    mae = mean_absolute_error(all_labels, all_preds)
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))

    return avg_loss, accuracy, mae, all_preds, all_labels, all_probs

In [ ]:
print("\n" + "="*70)
print("IMPROVED TRAINING WITH BETTER TECHNIQUES")
print("="*70 + "\n")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# BETTER HYPERPARAMETERS
MAX_LENGTH = 256
BATCH_SIZE = 16  # Larger batch with gradient accumulation
ACCUMULATION_STEPS = 2  # Effective batch size = 16 * 2 = 32

print("\nCreating datasets...")
train_dataset = CSSRSDataset(train_df['text'].values, train_df['label_id'].values, tokenizer, MAX_LENGTH)
val_dataset = CSSRSDataset(val_df['text'].values, val_df['label_id'].values, tokenizer, MAX_LENGTH)
test_dataset = CSSRSDataset(test_df['text'].values, test_df['label_id'].values, tokenizer, MAX_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"✓ Train batches: {len(train_loader)} (effective batch size: {BATCH_SIZE * ACCUMULATION_STEPS})")
print(f"✓ Val batches: {len(val_loader)}")
print(f"✓ Test batches: {len(test_loader)}")

# Initialize improved model
print(f"\nInitializing improved model...")
model = OrdinalCSSRSClassifier(MODEL_NAME, num_classes=5, dropout=0.2)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)")

# ============================================================
# RECALCULATE CLASS WEIGHTS - MORE MODERATE
# ============================================================

print("\n" + "="*70)
print("RECALCULATING CLASS WEIGHTS (More Balanced)")
print("="*70 + "\n")

# Use sqrt smoothing to make weights less aggressive
class_counts = train_df['label_id'].value_counts().sort_index()
total = len(train_df)

new_class_weights = []
for i in range(5):
    count = class_counts.get(i, 1)
    weight = np.sqrt(total / (5 * count))  # sqrt smoothing
    new_class_weights.append(weight)

new_class_weights = torch.FloatTensor(new_class_weights).to(device)
print("New class weights (sqrt-smoothed):")
for i, weight in enumerate(new_class_weights):
    print(f"  {id_to_label[i]:12s}: {weight:.3f} (count: {class_counts.get(i, 0)})")

# ============================================================
# IMPROVED TRAINING CONFIGURATION
# ============================================================

print("\n" + "="*70)
print("TRAINING CONFIGURATION")
print("="*70 + "\n")

NUM_EPOCHS = 20
LEARNING_RATE = 3e-5  # Slightly higher for better convergence
WARMUP_RATIO = 0.15
WEIGHT_DECAY = 0.01

print(f"Epochs: {NUM_EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Batch size: {BATCH_SIZE} (effective: {BATCH_SIZE * ACCUMULATION_STEPS})")
print(f"Gradient accumulation steps: {ACCUMULATION_STEPS}")
print(f"Warmup ratio: {WARMUP_RATIO}")

# Use Focal Loss instead of simple weighted CE
criterion = ImprovedOrdinalLoss(
    class_weights=new_class_weights,
    gamma=2.0,  # Focal loss parameter
    ordinal_weight=0.3
)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)

total_steps = len(train_loader) * NUM_EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# ============================================================
# TRAINING LOOP
# ============================================================

print("\n" + "="*70)
print("STARTING IMPROVED TRAINING")
print("="*70 + "\n")

best_val_mae = float('inf')
best_val_acc = 0
patience = 7  # More patience
patience_counter = 0
history = {
    'train_loss': [], 'train_acc': [], 'train_mae': [],
    'val_loss': [], 'val_acc': [], 'val_mae': []
}

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*70}")
    print(f"EPOCH {epoch + 1}/{NUM_EPOCHS}")
    print(f"{'='*70}\n")

    train_loss, train_acc, train_mae = train_epoch(
        model, train_loader, optimizer, scheduler, criterion, device, ACCUMULATION_STEPS
    )

    val_loss, val_acc, val_mae, val_preds, val_labels, _ = evaluate(
        model, val_loader, criterion, device
    )

    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['train_mae'].append(train_mae)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_mae'].append(val_mae)

    print(f"\nTrain Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | MAE: {train_mae:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | MAE: {val_mae:.4f}")

    # Print validation predictions distribution
    val_pred_dist = np.bincount(val_preds, minlength=5)
    print(f"Val predictions: {val_pred_dist}")

    # Save if better
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_val_acc = val_acc
        patience_counter = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_mae': val_mae,
            'val_acc': val_acc,
        }, 'best_cssrs_model_v2.pt')
        print(f"✓ New best MAE! Model saved.")
    else:
        patience_counter += 1
        print(f"Patience: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch + 1}")
            break

print(f"\n{'='*70}")
print("TRAINING COMPLETED")
print(f"{'='*70}")
print(f"Best validation MAE: {best_val_mae:.4f}")
print(f"Best validation accuracy: {best_val_acc:.4f}")

# ============================================================
# FINAL EVALUATION
# ============================================================

print("\n" + "="*70)
print("FINAL EVALUATION ON TEST SET")
print("="*70 + "\n")

checkpoint = torch.load('best_cssrs_model_v2.pt', weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✓ Loaded best model from epoch {checkpoint['epoch'] + 1}")

test_loss, test_acc, test_mae, test_preds, test_labels, test_probs = evaluate(
    model, test_loader, criterion, device
)

print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test MAE: {test_mae:.4f}")

print("\n" + "="*70)
print("CLASSIFICATION REPORT")
print("="*70 + "\n")

print(classification_report(
    test_labels,
    test_preds,
    target_names=[id_to_label[i] for i in range(5)],
    digits=4
))

print("\n" + "="*70)
print("CONFUSION MATRIX")
print("="*70 + "\n")

cm = confusion_matrix(test_labels, test_preds)
print("          ", " ".join([f"{id_to_label[i]:>10s}" for i in range(5)]))
for i, row in enumerate(cm):
    print(f"{id_to_label[i]:>10s}", " ".join([f"{val:>10d}" for val in row]))

ordinal_distances = np.abs(np.array(test_preds) - np.array(test_labels))
print(f"\nOrdinal Distance Analysis:")
print(f"  Exact match: {np.sum(ordinal_distances == 0)} ({np.mean(ordinal_distances == 0)*100:.1f}%)")
print(f"  Off by 1: {np.sum(ordinal_distances == 1)} ({np.mean(ordinal_distances == 1)*100:.1f}%)")
print(f"  Off by 2: {np.sum(ordinal_distances == 2)} ({np.mean(ordinal_distances == 2)*100:.1f}%)")
print(f"  Off by 3+: {np.sum(ordinal_distances >= 3)} ({np.mean(ordinal_distances >= 3)*100:.1f}%)")

print("\n" + "="*70)
print("PER-CLASS PERFORMANCE")
print("="*70 + "\n")

for i in range(5):
    mask = np.array(test_labels) == i
    if mask.sum() > 0:
        class_acc = np.mean(np.array(test_preds)[mask] == i)
        class_mae = mean_absolute_error(np.array(test_labels)[mask], np.array(test_preds)[mask])
        recall = class_acc
        support = mask.sum()
        print(f"{id_to_label[i]:12s}: Acc={class_acc:.3f}, MAE={class_mae:.3f}, Support={support:2d}")

print("\n" + "="*70)
print("MODEL READY FOR MALAYALAM TRANSFER")
print("="*70)


In [ ]:
# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("\nConfusion Matrix (rows=true, cols=predicted):")
print(cm)

# Ordinal confusion analysis
print("\n" + "="*70)
print("ORDINAL ERROR ANALYSIS")
print("="*70 + "\n")

errors = all_preds - all_labels
print(f"Exact matches:        {np.sum(errors == 0):3d} ({np.mean(errors == 0)*100:5.1f}%)")
print(f"Off by 1:             {np.sum(np.abs(errors) == 1):3d} ({np.mean(np.abs(errors) == 1)*100:5.1f}%)")
print(f"Off by 2:             {np.sum(np.abs(errors) == 2):3d} ({np.mean(np.abs(errors) == 2)*100:5.1f}%)")
print(f"Off by 3+:            {np.sum(np.abs(errors) >= 3):3d} ({np.mean(np.abs(errors) >= 3)*100:5.1f}%)")

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='RdYlGn_r',  # Red for errors, green for correct
    xticklabels=list(label_to_id.keys()),
    yticklabels=list(label_to_id.keys()),
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - Ordinal Suicidality Classification\n(Diagonal = Correct, Off-diagonal = Errors)',
          fontsize=14, pad=20)
plt.ylabel('True Severity Level', fontsize=12)
plt.xlabel('Predicted Severity Level', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix_ordinal.png', dpi=300, bbox_inches='tight')
print("\n✓ Confusion matrix saved as 'confusion_matrix_ordinal.png'")

# ============================================================
# 14. PER-CLASS DETAILED ANALYSIS
# ============================================================

print("\n" + "="*70)
print("PER-CLASS DETAILED PERFORMANCE")
print("="*70 + "\n")

for label_id in sorted(label_to_id.values()):
    label_name = id_to_label[label_id]

    # Get metrics for this class
    mask_true = all_labels == label_id
    mask_pred = all_preds == label_id

    true_count = mask_true.sum()
    pred_count = mask_pred.sum()

    if true_count > 0:
        # True positives
        tp = np.sum(mask_true & mask_pred)

        # Class-specific accuracy (how many of this class were correctly predicted)
        class_acc = tp / true_count if true_count > 0 else 0

        # F1 score
        f1 = f1_score(all_labels, all_preds, labels=[label_id], average='macro', zero_division=0)

        # Average predicted probability for this class
        avg_prob_true = all_probs[mask_true, label_id].mean() if true_count > 0 else 0

        # Where did misclassifications go?
        misclassified = all_preds[mask_true & ~mask_pred]

        print(f"\n{label_name.upper()} (Level {label_id})")
        print(f"  Support (test set):       {true_count:3d}")
        print(f"  Correctly predicted:      {tp:3d} / {true_count:3d} ({class_acc*100:5.1f}%)")
        print(f"  F1 Score:                 {f1:.4f}")
        print(f"  Avg confidence (correct): {avg_prob_true:.4f}")

        if len(misclassified) > 0:
            print(f"  Misclassification pattern:")
            for wrong_label_id, count in Counter(misclassified).most_common():
                wrong_label_name = id_to_label[wrong_label_id]
                distance = abs(wrong_label_id - label_id)
                print(f"    → {wrong_label_name:12s} (distance={distance}): {count:2d} samples")

# ============================================================
# 15. CRITICAL METRICS FOR ATTEMPT CLASS (MOST IMPORTANT)
# ============================================================

print("\n" + "="*70)
print("CRITICAL ANALYSIS: ATTEMPT CLASS (Most Severe, Only 45 samples)")
print("="*70 + "\n")

attempt_id = label_to_id['Attempt']
attempt_mask = all_labels == attempt_id
attempt_count = attempt_mask.sum()

if attempt_count > 0:
    attempt_preds = all_preds[attempt_mask]
    attempt_correct = (attempt_preds == attempt_id).sum()
    attempt_recall = attempt_correct / attempt_count

    print(f"Total Attempt samples in test set: {attempt_count}")
    print(f"Correctly identified:               {attempt_correct} ({attempt_recall*100:.1f}%)")
    print(f"Missed Attempts:                    {attempt_count - attempt_correct}")

    # Where were Attempts misclassified?
    missed_attempts = attempt_preds[attempt_preds != attempt_id]
    if len(missed_attempts) > 0:
        print(f"\nMisclassification breakdown:")
        for pred_id, count in Counter(missed_attempts).most_common():
            pred_name = id_to_label[pred_id]
            distance = abs(pred_id - attempt_id)
            risk = "HIGH RISK" if distance >= 2 else "Moderate"
            print(f"  → Predicted as {pred_name:12s} (distance={distance}, {risk}): {count} samples")

    # How many non-Attempts were falsely predicted as Attempts?
    non_attempt_mask = all_labels != attempt_id
    false_attempts = np.sum((all_preds[non_attempt_mask] == attempt_id))
    print(f"\nFalse Attempt predictions (false positives): {false_attempts}")

    # Precision for Attempt class
    if (attempt_preds == attempt_id).sum() + false_attempts > 0:
        attempt_precision = attempt_correct / ((attempt_preds == attempt_id).sum() + false_attempts)
        print(f"Attempt Precision: {attempt_precision:.4f}")
else:
    print("⚠ No Attempt samples in test set")

# ============================================================
# 16. ERROR DISTANCE DISTRIBUTION
# ============================================================

print("\n" + "="*70)
print("ERROR DISTANCE DISTRIBUTION (Ordinal Penalty Analysis)")
print("="*70 + "\n")

error_distances = np.abs(all_preds - all_labels)
plt.figure(figsize=(10, 6))
plt.hist(error_distances, bins=np.arange(-0.5, 5.5, 1), edgecolor='black', alpha=0.7)
plt.xlabel('Prediction Error Distance (|predicted - true|)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Ordinal Prediction Errors', fontsize=14)
plt.xticks(range(5))
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('error_distribution.png', dpi=300, bbox_inches='tight')
print("✓ Error distribution plot saved as 'error_distribution.png'")

for dist in range(5):
    count = np.sum(error_distances == dist)
    pct = count / len(error_distances) * 100
    severity = ["Perfect", "Minor", "Moderate", "Severe", "Critical"][dist]
    print(f"  Distance {dist} ({severity:8s}): {count:3d} ({pct:5.1f}%)")

# ============================================================
# 17. PROBABILITY CALIBRATION ANALYSIS
# ============================================================

print("\n" + "="*70)
print("PREDICTION CONFIDENCE ANALYSIS")
print("="*70 + "\n")

# Average confidence for correct vs incorrect predictions
correct_mask = all_preds == all_labels
incorrect_mask = ~correct_mask

correct_confidence = all_probs[correct_mask, all_preds[correct_mask]].mean()
incorrect_confidence = all_probs[incorrect_mask, all_preds[incorrect_mask]].mean()

print(f"Average confidence (correct predictions):   {correct_confidence:.4f}")
print(f"Average confidence (incorrect predictions): {incorrect_confidence:.4f}")
print(f"Confidence gap:                             {correct_confidence - incorrect_confidence:.4f}")

# Per-class average confidence
print("\nPer-class average prediction confidence:")
for label_id in sorted(label_to_id.values()):
    label_name = id_to_label[label_id]
    mask = all_preds == label_id
    if mask.sum() > 0:
        avg_conf = all_probs[mask, label_id].mean()
        print(f"  {label_name:12s}: {avg_conf:.4f}")

# ============================================================
# 18. SAVE MODEL AND ARTIFACTS
# ============================================================

print("\n" + "="*70)
print("SAVING MODEL AND ARTIFACTS")
print("="*70 + "\n")

# Save MURIL encoder
model.muril.save_pretrained('./muril_suicide_ordinal_final')
tokenizer.save_pretrained('./muril_suicide_ordinal_final')

# Save full model state dict
torch.save(model.state_dict(), './muril_suicide_ordinal_final/full_model.pt')

# Save label mappings and metadata
import json

metadata = {
    'label_to_id': label_to_id,
    'id_to_label': id_to_label,
    'num_labels': NUM_LABELS,
    'max_length': MAX_LENGTH,
    'model_name': MODEL_NAME,
    'class_weights': class_weights.cpu().numpy().tolist(),
    'training_info': {
        'total_samples': len(df),
        'train_samples': len(train_df),
        'val_samples': len(val_df),
        'test_samples': len(test_df),
        'label_distribution': label_counts.to_dict()
    },
    'final_metrics': {
        'accuracy': float(accuracy_score(all_labels, all_preds)),
        'f1_macro': float(f1_score(all_labels, all_preds, average='macro', zero_division=0)),
        'mae': float(mean_absolute_error(all_labels, all_preds)),
        'qwk': float(cohen_kappa_score(all_labels, all_preds, weights='quadratic')),
        'off_by_one_acc': float(np.mean(np.abs(all_preds - all_labels) <= 1))
    }
}

with open('./muril_suicide_ordinal_final/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✓ MURIL encoder saved to './muril_suicide_ordinal_final'")
print("✓ Full model state_dict saved")
print("✓ Tokenizer saved")
print("✓ Metadata and label mappings saved")

# ============================================================
# 19. EXAMPLE PREDICTIONS WITH CONFIDENCE
# ============================================================

print("\n" + "="*70)
print("EXAMPLE PREDICTIONS (with confidence scores)")
print("="*70 + "\n")

# Sample predictions from each severity level
for target_label_id in sorted(label_to_id.values()):
    target_label_name = id_to_label[target_label_id]
    target_indices = test_df[test_df['label_id'] == target_label_id].index.tolist()

    if len(target_indices) > 0:
        # Get one example from this class
        sample_idx = np.random.choice(len(target_indices))
        df_idx = target_indices[sample_idx]

        text = test_df.loc[df_idx, 'text']
        true_label = test_df.loc[df_idx, 'label']

        # Tokenize and predict
        encoding = tokenizer(
            text,
            max_length=MAX_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        model.eval()
        with torch.no_grad():
            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            logits = outputs['logits']
            probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
            pred_id = np.argmax(probs)
            pred_label = id_to_label[pred_id]

        print(f"\n{'-'*70}")
        print(f"TRUE LABEL: {true_label.upper()} (Level {target_label_id})")
        print(f"PREDICTED:  {pred_label.upper()} (Level {pred_id})")
        print(f"Text: {text[:150]}..." if len(text) > 150 else f"Text: {text}")
        print(f"\nPrediction probabilities:")
        for lid in sorted(label_to_id.values()):
            lname = id_to_label[lid]
            prob = probs[lid]
            bar = '█' * int(prob * 40)
            print(f"  {lname:12s} (L{lid}): {prob:.4f} {bar}")

# ============================================================
# 20. CLINICAL RECOMMENDATIONS
# ============================================================

print("\n" + "="*70)
print("CLINICAL DEPLOYMENT RECOMMENDATIONS")
print("="*70 + "\n")

print("""
1. THRESHOLD ADJUSTMENT:
   - Consider using probability thresholds rather than argmax
   - For "Attempt" class (most severe), use lower threshold to increase recall
   - Example: Flag as "Attempt" if P(Attempt) > 0.3 instead of 0.5

2. ORDINAL CONFIDENCE:
   - Predictions off by 1 level (e.g., Behavior→Attempt) are acceptable
   - Focus on preventing large errors (e.g., Supportive→Attempt)

3. HUMAN REVIEW TRIGGERS:
   - Auto-flag all predictions of Behavior or Attempt for human review
   - Flag predictions where top 2 probabilities are close (uncertain cases)

4. MODEL LIMITATIONS:
   - Small dataset (500 samples), especially for Attempt class (45 samples)
   - Model may underpredict severe cases due to class imbalance
   - NOT a replacement for professional mental health assessment

5. ETHICAL CONSIDERATIONS:
   - This is a screening tool, not a diagnostic tool
   - Must be used alongside human clinical judgment
   - Privacy and data security are paramount
   - Consider impact of false negatives (missing severe cases)

6. PERFORMANCE SUMMARY:
   - Best at identifying Ideation (most common class)
   - Struggles with Attempt (rarest class, only 9% of data)
   - Ordinal structure helps reduce catastrophic errors
   - Off-by-one accuracy shows model respects severity ordering
""")

print("="*70)
print("TRAINING COMPLETE - MODEL READY FOR EVALUATION")
print("="*70)

# ============================================================
# 21. SAVE INFERENCE SCRIPT
# ============================================================

inference_script = '''
# INFERENCE SCRIPT FOR MURIL ORDINAL SUICIDE CLASSIFICATION
import torch
from transformers import AutoTokenizer, AutoModel
import json
import numpy as np

# Load model and metadata
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('./muril_suicide_ordinal_final')

with open('./muril_suicide_ordinal_final/metadata.json', 'r') as f:
    metadata = json.load(f)

id_to_label = {int(k): v for k, v in metadata['id_to_label'].items()}

# Recreate model architecture
from your_training_script import MURILForOrdinalSuicideClassification
model = MURILForOrdinalSuicideClassification(
    metadata['model_name'],
    metadata['num_labels'],
    dropout=0.3
)
model.load_state_dict(torch.load('./muril_suicide_ordinal_final/full_model.pt'))
model.to(device)
model.eval()

def predict_suicidality(text, threshold=0.5):
    """
    Predict suicidality level for given text

    Args:
        text: Input text (Reddit post, message, etc.)
        threshold: Probability threshold for high-risk flagging

    Returns:
        dict with prediction, probabilities, and risk level
    """
    encoding = tokenizer(
        text,
        max_length=metadata['max_length'],
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    with torch.no_grad():
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask)
        logits = outputs['logits']
        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

    pred_id = np.argmax(probs)
    pred_label = id_to_label[pred_id]
    confidence = probs[pred_id]

    # Risk assessment
    risk_level = "LOW" if pred_id <= 1 else "MODERATE" if pred_id == 2 else "HIGH"

    # Flag for human review if high severity or uncertain
    needs_review = (pred_id >= 3) or (confidence < 0.6)

    return {
        'predicted_level': pred_id,
        'predicted_label': pred_label,
        'confidence': float(confidence),
        'risk_level': risk_level,
        'needs_human_review': needs_review,
        'all_probabilities': {id_to_label[i]: float(probs[i]) for i in range(len(probs))}
    }

# Example usage
if __name__ == "__main__":
    test_text = "I don't see the point anymore. Everything feels hopeless."
    result = predict_suicidality(test_text)
    print(f"Prediction: {result['predicted_label']} (Level {result['predicted_level']})")
    print(f"Confidence: {result['confidence']:.4f}")
    print(f"Risk Level: {result['risk_level']}")
    print(f"Needs Review: {result['needs_human_review']}")
'''

with open('./muril_suicide_ordinal_final/inference.py', 'w') as f:
    f.write(inference_script)

print("\n✓ Inference script saved to './muril_suicide_ordinal_final/inference.py'")
print("\nAll artifacts saved. Model ready for deployment.")
print("="*70)

# Comprehensive Depression score predictor (Idea seems wrong. checking...)

In [ ]:
!pip install  xgboost optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/PHQ9-Malayalam/PHQ-9_Dataset_5th Edition.csv')

print("Dataset shape:", df.shape)
print("\nDataset info:")
print(df.info())
print("\nMissing values:")
print(df.isnull().sum())


Dataset shape: (682, 16)

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 16 columns):
 #   Column                                                                                                                                                                    Non-Null Count  Dtype 
---  ------                                                                                                                                                                    --------------  ----- 
 0   Age                                                                                                                                                                       682 non-null    int64 
 1   Gender                                                                                                                                                                    682 non-null    object
 2   Little interest or pleasure in doing things               

In [ ]:
# ============================================
# DATA PREPROCESSING
# ============================================

# Clean column names (remove extra spaces)
df.columns = df.columns.str.strip()

print("\n--- Debug: After column name strip ---")
print(f"Shape: {df.shape}")

# Handle missing values - initial dropna
df = df.dropna()
print("\n--- Debug: After initial df.dropna() ---")
print(f"Shape: {df.shape}")
print(df.isnull().sum())

# Define PHQ-9 symptom columns
phq_symptoms = [
    'Little interest or pleasure in doing things',
    'Feeling down, depressed, or hopeless',
    'Trouble falling or staying asleep, or sleeping too much',
    'Feeling tired or having little energy',
    'Poor appetite or overeating',
    'Feeling bad about yourself—or that you are a failure or have let yourself or your family down',
    'Trouble concentrating on things, such as reading the newspaper or watching television',
    'Moving or speaking so slowly that other people could have noticed? Or the opposite—being so fidgety or restless that you have been moving around a lot more than usual',
    'Thoughts that you would be better off dead or of hurting yourself in some way'
]

# Create ordinal mapping for PHQ-9 symptoms
symptom_mapping = {
    'Not at all': 0,
    'Several days': 1,
    'More than half the days': 2,
    'Nearly every day': 3
}

# Encode PHQ-9 symptoms
for col in phq_symptoms:
    if col in df.columns:
        print(f"\n--- Debug: Unique values in '{col}' before mapping ---")
        print(df[col].astype(str).str.strip().unique())
        df[col] = df[col].astype(str).str.strip().map(symptom_mapping)
print("\n--- Debug: After PHQ-9 symptom mapping ---")
print(f"Shape: {df.shape}")
print(df.isnull().sum())

# Encode categorical variables
# Gender
gender_mapping = {'Male': 0, 'Female': 1, 'Other': 2}
print("\n--- Debug: Unique values in 'Gender' before mapping ---")
print(df['Gender'].astype(str).str.strip().unique())
df['Gender'] = df['Gender'].astype(str).str.strip().map(gender_mapping)

# Sleep Quality
sleep_mapping = {'Good': 0, 'Average': 1, 'Bad': 2, 'Worst': 3}
print("\n--- Debug: Unique values in 'Sleep Quality' before mapping ---")
print(df['Sleep Quality'].astype(str).str.strip().unique())
df['Sleep Quality'] = df['Sleep Quality'].astype(str).str.strip().map(sleep_mapping)

# Study Pressure
study_mapping = {'Good': 0, 'Average': 1, 'Bad': 2}
print("\n--- Debug: Unique values in 'Study Pressure' before mapping ---")
print(df['Study Pressure'].astype(str).str.strip().unique())
df['Study Pressure'] = df['Study Pressure'].astype(str).str.strip().map(study_mapping)

# Financial Pressure
financial_mapping = {'Good': 0, 'Average': 1, 'Bad': 2}
print("\n--- Debug: Unique values in 'Financial Pressure' before mapping ---")
print(df['Financial Pressure'].astype(str).str.strip().unique())
df['Financial Pressure'] = df['Financial Pressure'].astype(str).str.strip().map(financial_mapping)

print("\n--- Debug: After all categorical mappings ---")
print(f"Shape: {df.shape}")
print(df.isnull().sum())

# Remove PHQ_Severity as it's derived from PHQ_Total
if 'PHQ_Severity' in df.columns:
    df = df.drop('PHQ_Severity', axis=1)

# Ensure all columns are numeric after mapping, dropping rows with any NaNs
# that might have been introduced by .map() if a value was not in the dictionary.
print("\n--- Debug: Before final df.dropna() ---")
print(f"Shape: {df.shape}")
print(df.isnull().sum())
df = df.dropna().reset_index(drop=True)

print("\nData after preprocessing:")
print(df.head())
print("\nData types:")
print(df.dtypes)



--- Debug: After column name strip ---
Shape: (682, 16)

--- Debug: After initial df.dropna() ---
Shape: (682, 16)
Age                                                                                                                                                                       0
Gender                                                                                                                                                                    0
Little interest or pleasure in doing things                                                                                                                               0
Feeling down, depressed, or hopeless                                                                                                                                      0
Trouble falling or staying asleep, or sleeping too much                                                                                                                   0
Feeling tired or having 

In [ ]:
# ============================================
# FEATURE ENGINEERING
# ============================================

# Create additional features
df['Total_Symptoms_Score'] = df[phq_symptoms].sum(axis=1)
df['Avg_Symptom_Severity'] = df[phq_symptoms].mean(axis=1)
df['Severe_Symptoms_Count'] = (df[phq_symptoms] >= 2).sum(axis=1)
df['External_Stress_Score'] = df['Sleep Quality'] + df['Study Pressure'] + df['Financial Pressure']

# Interaction features
df['Sleep_Study_Interaction'] = df['Sleep Quality'] * df['Study Pressure']
df['Age_Gender_Interaction'] = df['Age'] * df['Gender']

In [ ]:
# ============================================
# PREPARE DATA FOR MODELING
# ============================================

# Separate features and target
X = df.drop('PHQ_Total', axis=1)
y = df['PHQ_Total']

# Add a final check for NaNs after feature engineering
# and before splitting and scaling, just to be safe.
# This also handles any potential NaNs created during feature engineering
# (though unlikely with the current operations).
combined_df = pd.concat([X, y], axis=1)
combined_df = combined_df.dropna().reset_index(drop=True)
X = combined_df.drop('PHQ_Total', axis=1)
y = combined_df['PHQ_Total']


# Identify and remove zero-variance columns before scaling
# These columns cause StandardScaler to produce NaNs
zero_variance_cols = X.columns[X.var() == 0]
if not zero_variance_cols.empty:
    print(f"Warning: Removing zero-variance columns: {list(zero_variance_cols)}")
    X = X.drop(columns=zero_variance_cols)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nTraining set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")


Training set size: 402
Test set size: 101


In [ ]:
# ============================================
# OPTUNA OPTIMIZATION
# ============================================

def objective(trial):
    """Objective function for Optuna optimization"""

    # Suggest which model to use
    model_name = trial.suggest_categorical('model', ['xgboost', 'random_forest', 'gradient_boosting', 'ridge'])

    if model_name == 'xgboost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
            'gamma': trial.suggest_float('gamma', 0, 0.5),
            'random_state': 42
        }
        model = XGBRegressor(**params)

    elif model_name == 'random_forest':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'random_state': 42
        }
        model = RandomForestRegressor(**params)

    elif model_name == 'gradient_boosting':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'random_state': 42
        }
        model = GradientBoostingRegressor(**params)

    else:  # ridge
        params = {
            'alpha': trial.suggest_float('alpha', 0.01, 100, log=True),
            'random_state': 42
        }
        model = Ridge(**params)

    # Cross-validation score
    cv_scores = cross_val_score(
        model, X_train_scaled, y_train,
        cv=5, scoring='neg_mean_squared_error'
    )

    return -cv_scores.mean()  # Return negative MSE (Optuna minimizes)

In [ ]:
# Run Optuna optimization
print("\n" + "="*50)
print("STARTING OPTUNA OPTIMIZATION")
print("="*50)

study = optuna.create_study(direction='minimize', study_name='depression_prediction')
study.optimize(objective, n_trials=100, show_progress_bar=True)

print("\n" + "="*50)
print("OPTIMIZATION COMPLETE")
print("="*50)
print(f"\nBest trial value (MSE): {study.best_trial.value:.4f}")
print(f"Best trial RMSE: {np.sqrt(study.best_trial.value):.4f}")
print("\nBest hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# ============================================
# TRAIN FINAL MODEL WITH BEST PARAMETERS
# ============================================

best_params = study.best_params.copy()
model_name = best_params.pop('model')

print(f"\n{'='*50}")
print(f"TRAINING FINAL MODEL: {model_name.upper()}")
print(f"{'='*50}")

if model_name == 'xgboost':
    best_params['random_state'] = 42
    final_model = XGBRegressor(**best_params)
elif model_name == 'random_forest':
    best_params['random_state'] = 42
    final_model = RandomForestRegressor(**best_params)
elif model_name == 'gradient_boosting':
    best_params['random_state'] = 42
    final_model = GradientBoostingRegressor(**best_params)
else:
    best_params['random_state'] = 42
    final_model = Ridge(**best_params)

# Train final model
final_model.fit(X_train_scaled, y_train)

[I 2025-12-09 17:10:23,412] A new study created in memory with name: depression_prediction



STARTING OPTUNA OPTIMIZATION


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-12-09 17:10:23,567] Trial 0 finished with value: 0.079487609547757 and parameters: {'model': 'ridge', 'alpha': 64.2705380437855}. Best is trial 0 with value: 0.079487609547757.
[I 2025-12-09 17:10:25,339] Trial 1 finished with value: 0.05394525537267327 and parameters: {'model': 'xgboost', 'n_estimators': 76, 'max_depth': 9, 'learning_rate': 0.29096725182074706, 'subsample': 0.7871400758635652, 'colsample_bytree': 0.8774559767702512, 'min_child_weight': 7, 'gamma': 0.06873386864160907}. Best is trial 1 with value: 0.05394525537267327.
[I 2025-12-09 17:10:26,185] Trial 2 finished with value: 0.10247377008199691 and parameters: {'model': 'xgboost', 'n_estimators': 299, 'max_depth': 10, 'learning_rate': 0.07516633620408593, 'subsample': 0.9869649454593658, 'colsample_bytree': 0.6947175630290858, 'min_child_weight': 6, 'gamma': 0.4441137980438356}. Best is trial 1 with value: 0.05394525537267327.
[I 2025-12-09 17:10:26,248] Trial 3 finished with value: 2.5907248258146418e-06 and pa

Ridge(alpha=0.010012729657070419, random_state=42)

In [ ]:
# ============================================
# MODEL EVALUATION
# ============================================

# Predictions
y_train_pred = final_model.predict(X_train_scaled)
y_test_pred = final_model.predict(X_test_scaled)

# Calculate metrics
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"\n{'='*50}")
print("MODEL PERFORMANCE")
print(f"{'='*50}")
print("\nTraining Set:")
print(f"  RMSE: {train_rmse:.4f}")
print(f"  MAE:  {train_mae:.4f}")
print(f"  R²:   {train_r2:.4f}")

print("\nTest Set:")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  MAE:  {test_mae:.4f}")
print(f"  R²:   {test_r2:.4f}")

# Feature importance (if available)
if hasattr(final_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': final_model.feature_importances_
    }).sort_values('Importance', ascending=False)

    print(f"\n{'='*50}")
    print("TOP 10 FEATURE IMPORTANCES")
    print(f"{'='*50}")
    print(feature_importance.head(10).to_string(index=False))

# Sample predictions
print(f"\n{'='*50}")
print("SAMPLE PREDICTIONS")
print(f"{'='*50}")
sample_results = pd.DataFrame({
    'Actual': y_test.values[:10],
    'Predicted': y_test_pred[:10],
    'Difference': y_test.values[:10] - y_test_pred[:10]
})
print(sample_results.to_string(index=False))




MODEL PERFORMANCE

Training Set:
  RMSE: 0.0001
  MAE:  0.0001
  R²:   1.0000

Test Set:
  RMSE: 0.0001
  MAE:  0.0001
  R²:   1.0000

SAMPLE PREDICTIONS
 Actual  Predicted  Difference
     18  18.000084   -0.000084
      0   0.000130   -0.000130
      7   6.999848    0.000152
     12  11.999976    0.000024
      0   0.000167   -0.000167
      9   9.000035   -0.000035
     22  22.000028   -0.000028
      4   3.999891    0.000109
      0   0.000166   -0.000166
      6   6.000095   -0.000095
